<a href="https://colab.research.google.com/github/silveira-alysson/Strategic-Intelligence-LLM/blob/main/Decoda_SignOfLife_24Jan24_v5aos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Deep Active Learning Component**

Inputs:
1. Corpus (10k reports as a dictionary)
2. List of reports that were used in the test set (manually coded)

Process:
1. Import corpus
2. Exclude subsample used in manual labeling
3. Encode remaining documents
4. Calculate similarity with ontology
5. Choose top and bottom 200 documents in terms of similarity

Outputs:
1. Tokened corpus (pickle) - Intermediary output
2. Enconded corpus (pickle) - Intermediary output
3. File for human disambiguation

In [ ]:
!pip install sentence-transformers --quiet
import pickle
import pandas as pd
import numpy as np
import os
from sentence_transformers import SentenceTransformer, util

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.0 MB/s eta 0:00:00


In [ ]:
if __name__ == "__main__":
  #import whole 10k
  encoded_corpus= pickle.load(open('/work_bgfs/a/alysson/Embedded_10k_whole/cdict.pickle', "rb"))

  #import test set to check overlap
  test = pd.read_csv('/work_bgfs/a/alysson/Embedded_10k_whole/test_set_assension_numbers.csv')

  # Drop observations that are part of the test set
  for unwanted_key in test['Row Labels']: del encoded_corpus[unwanted_key]

  #generate a random sample of 3000 keys
  keys = list(pd.DataFrame(encoded_corpus.keys(), columns=['key']).sample(3000, random_state=17).key)

  encoded_corpus =  {k:v for (k,v) in encoded_corpus.items() if k in keys}

  #import sentence transformers
  model_save_path = 'all-roberta-large-v1'
  model = SentenceTransformer(model_save_path)

  # encode the ontology
  query_embedding = model.encode(["introduced a new generation of products",
        "extended the product range", "openeded new markets",
        "entered a new technology field" ,"improved an existing product quality",
        "improved production flexibility", "reduced production cost",
        "improved yield or reduced material consumption"])

  # create data frame
  df = pd.DataFrame(columns = ['key', 'sentence', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8'])
  for k, v in encoded_corpus.items():
    if v != []:
      df1 = pd.DataFrame([np.repeat(k ,len(v[0])), v[0], *util.cos_sim(query_embedding, v[1]).numpy()]).T
      df1.columns = ['key', 'sentence', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8']
      df = pd.concat((df, df1), axis=0)

  pickle.dump(df, open('/work_bgfs/a/alysson/Embedded_10k_whole/Sample3000DF.pickle', "wb"))


In [ ]:
  df = pickle.load(open('/work_bgfs/a/alysson/Embedded_10k_whole/Sample3000DF.pickle', "rb"))
  #create sentence id
  df['sentenceID'] = (df.groupby(['key']).cumcount()).astype(int)

  #drop duplicated sentences
  df = df.drop_duplicates(subset='sentence', keep='first', inplace=False)

  #sort by similarity score within categories and display top 500
  topC1 = df.sort_values(['c1'],ascending=False).head(500).sample(100)
  topC2 = df.sort_values(['c2'],ascending=False).head(500).sample(100)
  topC3 = df.sort_values(['c3'],ascending=False).head(500).sample(100)
  topC4 = df.sort_values(['c4'],ascending=False).head(500).sample(100)
  topC5 = df.sort_values(['c5'],ascending=False).head(500).sample(100)
  topC6 = df.sort_values(['c6'],ascending=False).head(500).sample(100)
  topC7 = df.sort_values(['c7'],ascending=False).head(500).sample(100)
  topC8 = df.sort_values(['c8'],ascending=False).head(500).sample(100)

  topSentences = topC1.append(topC2).append(topC3).append(topC4).append(topC5).append(topC6).append(topC7).append(topC8)
  pickle.dump(topSentences , open('/work_bgfs/a/alysson/Embedded_10k_whole/top500sample100sentencesPerCategory3000Sample.pickle', "wb"))
  topSentences.to_excel('/work_bgfs/a/alysson/Embedded_10k_whole/top500sample100sentencesPerCategory3000Sample.xlsx')
  pickle.dump(df, open('/work_bgfs/a/alysson/Embedded_10k_whole/finalSampleDF_V2.pickle', "wb"))

FileNotFoundError: [Errno 2] No such file or directory: '/work_bgfs/a/alysson/Embedded_10k_whole/Sample3000DF.pickle'

In [ ]:
# Connect with Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Download corpus
!wget -nc -O cp_10k.pickle https://www.dropbox.com/s/3g8x0695zp6h5oc/cp_10k.pickle?dl=0

--2023-10-03 19:58:30--  https://www.dropbox.com/s/3g8x0695zp6h5oc/cp_10k.pickle?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/3g8x0695zp6h5oc/cp_10k.pickle [following]
--2023-10-03 19:58:30--  https://www.dropbox.com/s/raw/3g8x0695zp6h5oc/cp_10k.pickle
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd306ffe3d731e86d81a28a174d.dl.dropboxusercontent.com/cd/0/inline/CE5H6qoFFm6QNOR8ED-9JB-rpxL-oSD3oADcsdnKZiNdeIJ1nK2Np9ulMDHEQ-Vvexei5kp6jtt18tWACOWD2OZkjVfC1MltLRao39PnhiTCGV0eNhExCCMdxe0PNsWIKAZxdBEpL1l3vg4pcDbxSLbb/file# [following]
--2023-10-03 19:58:31--  https://ucd306ffe3d731e86d81a28a174d.dl.dropboxusercontent.com/cd/0/inline/CE5H6qoFFm6QNOR8ED-9JB-rpxL-oSD3oADcsdnKZiNdeIJ1nK2Np9ulMDHEQ-Vvexei5kp6jtt18tWACOWD2OZkjVf

In [ ]:
# Install sentence transformers
!pip install sentence-transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.2 MB/s eta 0:00:00


In [ ]:
#Import packages

import pickle
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import nltk
nltk.download('punkt')
from nltk import tokenize
import numpy as np
import sys
import os
import time
import itertools
os.environ["TOKENIZERS_PARALLELISM"] = "false"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#import corpus (Dictionary with Business section of 10k reports)
cp = pickle.load(open('/content/cp_10k.pickle', 'rb'))

In [ ]:
# Check lenght of cp
len(cp)

13270

In [ ]:
#import test set to check overlap
test = pd.read_excel('/content/drive/MyDrive/DCAST/Financial/Inputs/manual_coding_firm_level.xlsx').fillna(0)

In [ ]:
# Drop observations that are part of the test set
for unwanted_key in test['Row Labels']: del cp[unwanted_key]
len(cp)

In [ ]:
# sentence tokenize sample
tokenized_sample = {k :tokenize.sent_tokenize(v) for (k, v) in cp.items()}

In [ ]:
# save tokenized sample
pickle.dump(tokenized_sample, open('/content/drive/MyDrive/DCAST/Financial/Inputs/tokenized_corpus.pickle', 'wb'))


In [ ]:
#load transformer
model_save_path = 'all-roberta-large-v1'
model = SentenceTransformer(model_save_path)

encoded_corpus = {k :model.encode(v) for (k, v) in tokenized_sample.items()}

In [ ]:
# save encoded corpus
pickle.dump(encoded_corpus, open('/content/drive/MyDrive/DCAST/Financial/Inputs/encoded_corpus.pickle', 'wb'))

In [ ]:
# encode the ontology
query_embedding = model.encode(["introduced a new generation of products",
      "extended the product range", "openeded new markets",
      "entered a new technology field" ,"improved an existing product quality",
      "improved production flexibility", "reduced production cost",
      "improved yield or reduced material consumption"])

In [ ]:
# create data frame
df = pd.DataFrame(columns = ['key', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8'])
for k, v in encoded_corpus.items():
  if v != []:
    df1 = pd.DataFrame([np.repeat(k ,v.shape[0]), *util.cos_sim(query_embedding, v).numpy()]).T
    df1.columns = ['key', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8']
    df = pd.concat((df, df1), axis=0)

In [ ]:
#add sentences
dfv = pd.DataFrame()
for k, v in tokenized_sample.items():
  if v != []:
    dfv1 = pd.DataFrame(v)
    dfv = pd.concat((dfv, dfv1), axis=0)

In [ ]:

df = pd.concat([df, dfv], axis=1)
df = df.rename(columns={0:'sentence'})

In [ ]:
#generate sentence ids
df['sentenceID'] = (df.groupby(['key']).cumcount()).astype(int)

In [ ]:
# save data frame
pickle.dump(df, open('/content/drive/MyDrive/DCAST/Financial/Inputs/df.pickle', 'wb'))

In [ ]:
#re-create sentence ids because pickled version was wrong
df['sentenceID'] = (df.groupby(['key']).cumcount()).astype(int)

In [ ]:
for i in range(1,9):
  print(len(df[df["c"+str(i)]>0.47]))

In [ ]:
#generate excel sheet for human disambiguation
subsample = pd.DataFrame(columns = df.columns)
for i in range(1,9):
  a = df[df["c"+str(i)]>np.mean(df["c"+str(i)])+(3*np.std(df["c"+str(i)]))].reset_index(drop=True).sample(100)
  subsample = pd.concat((subsample, a), axis=0)

new_column = [np.repeat('c1', 100), np.repeat('c2', 100), np.repeat('c3', 100), np.repeat('c4', 100), np.repeat('c5', 100), np.repeat('c6', 100), np.repeat('c7', 100), np.repeat('c8', 100)]
subsample['class'] = list(itertools.chain(*new_column))
subsample = subsample.drop(['c1','c2','c3','c4','c5','c6','c7','c8'], axis = 1).reset_index(drop=True).reset_index(drop=True)

In [ ]:
# save file for human disambiguation
subsample.to_excel('/content/drive/MyDrive/MISQ/Intermediates/2.disambiguation_file.xlsx')

**Text Augmentation Component**

Inputs:
1. Disambiaguation_file.xlsx (humanly disambiguated examples)
2. Augmentation prompt templates

Process:
1. Pass augmentation prompts and disambiguated sentences to the OpenAI API to get completions

Output:
1. Augmented data frame


In [ ]:
# install and import packages
!pip install openai --quiet

import pandas as pd
import os
import time
import os
import openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
# pass Open AI API key
api_key = open("/content/drive/MyDrive/DCAST/Financial/Inputs/openAI_api.txt", "r")
openai.api_key = api_key.readlines(1)[0]

In [ ]:
#import disambuguated examples
# disambiguated_file = pd.read_excel('/content/drive/MyDrive/DCAST/Financial/Inputs/2.disambiguation_file_13Oct23_v1aos.xlsx', index_col=0)

# #filter augmentedDF to keep only the sentences labeled as positives in the human disambiguation component
# #this ensures that the negatives generated by the augmentation component are actual negatives
# disambiguated_file_positives = disambiguated_file[disambiguated_file['label']==1]

import ast
disambiguated_file_positives = pd.read_excel('/content/drive/MyDrive/DCAST/Financial/Inputs/2.disambiguation_file_13Oct23_v2aos.xlsx', index_col=0, sheet_name="Sheet2")
#disambiguated_file_positives = disambiguated_file_positives['d'].apply(ast.literal_eval).tolist()

In [ ]:
#import templates that will serve as examples for the prompt-based augmentation
templates = pd.read_excel('/content/drive/MyDrive/DCAST/Financial/Inputs/5.Augmentation_Prompt_Templates_10_Shots_6_Types_Corpus_Based_13Oct23_v2aos.xlsx')

In [ ]:
#Generate instructions and examples for each class

#create empty dataframe
augmentedDF = pd.DataFrame(columns=['key','sentenceID','anchor','category','typeAug','text'])

prompts = disambiguated_file_positives['prompt'].apply(ast.literal_eval).tolist()


contexts = ""
for j in range(0, len(disambiguated_file_positives.sentence)):
#for j in range(221, len(disambiguated_file_positives.sentence)):
  anchor = str(disambiguated_file_positives.iloc[j,1])
  c = [prompts[j]]
  key = str(disambiguated_file_positives.iloc[j,0])
  id = str(disambiguated_file_positives.iloc[j,2])
  cat = str(disambiguated_file_positives.iloc[j,3])

  for i in range(0, len(templates.Type0)):
    typeAug = templates.iloc[i,0] + "-" + templates.iloc[i,1]
    temp = templates["Text2"].apply(ast.literal_eval)[i]
    prompt = temp + c
    response = openai.ChatCompletion.create(
      model="gpt-4",
      messages=prompt
    )
    augmentedDF.loc[len(augmentedDF)] = [key, id, anchor, cat, typeAug, response['choices'][0]['message']['content']]
    time.sleep(7)
    if j % 10 == 0:
      augmentedDF.to_excel('/content/drive/MyDrive/DCAST/Financial/Inputs/6.augmentedDF_GPT4_24Oct23_v2aos.xlsx')

augmentedDF.to_excel('/content/drive/MyDrive/DCAST/Financial/Inputs/6.augmentedDF_GPT4_24Oct23_v2aos.xlsx')

### **Generate final dataset**

Inputs:
1. Augmented dataset
2. Humanly disambiguated file

Process:


Output:
1.

In [ ]:
# import datasets
import pandas as pd
import pickle
import numpy as np

In [ ]:
# Load augmented dataset
augmentedDF = pd.read_excel('//content/drive/MyDrive/DCAST/Financial/Inputs/6.augmentedDF_GPT4_24Oct23_v2aos.xlsx', index_col=0)

In [ ]:
#create label column based on the augType
w = pd.DataFrame.from_dict({'Entailment-Core args' : 1, 'Entailment-Redudancy': 2, 'Entailment-Nominalization': 3,
                            'Contradiction-Lexical': 4, 'Contradiction-Negation': 5, 'Contradiction-Random': 6}
                           , orient = 'index', columns=['classAug'])
augmentedDF = pd.merge(augmentedDF, w, left_on= 'typeAug', right_index= True)

In [ ]:
#create ontology column
o = pd.DataFrame.from_dict({"c1": "introduced a new generation of products", "c2":"extended the product range", "c3" : "opened new markets",
      "c4":"entered a new technology field" , "c5": "improved an existing product quality", "c6": "improved production flexibility",
      "c7": "reduced production cost", "c8": "improved yield or reduced material consumption"}, orient = 'index', columns=['ontology'])
augmentedDF = pd.merge(augmentedDF, o, left_on= 'category', right_index= True)

In [ ]:
#import human disambiguated file
disamb =  pd.read_excel('/content/drive/MyDrive/DCAST/Financial/Inputs/2.disambiguation_file_13Oct23_v2aos.xlsx', index_col=False).drop(columns="Unnamed: 0", axis=1).rename(columns={'class':'category'})

#generate unique id by merging sentenceID and class
disamb['idCat'] = None
for i in range(0, len(disamb)):
  disamb['idCat'][i] = disamb['key'][i] +'-'+ str(int(disamb['sentenceID'][i])) + '-' + disamb['category'][i]

<ipython-input-7-9532d14bc6bc>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disamb['idCat'][i] = disamb['key'][i] +'-'+ str(int(disamb['sentenceID'][i])) + '-' + disamb['category'][i]


In [ ]:
#generate unique id by merging sentenceID and category
augmentedDF['idCat'] = None
for i in range(0, len(augmentedDF)):
  augmentedDF['idCat'][i] = augmentedDF['key'][i] +'-'+ str(int(augmentedDF['sentenceID'][i])) + '-' + augmentedDF['category'][i]

<ipython-input-8-db4275a745e2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  augmentedDF['idCat'][i] = augmentedDF['key'][i] +'-'+ str(int(augmentedDF['sentenceID'][i])) + '-' + augmentedDF['category'][i]


In [ ]:
#filter augmentedDF to keep only the sentences labeled as positives in the human disambiguation component
#this ensures that the negatives generated by the augmentation component are actual negatives
#positives = augmentedDF[augmentedDF['idCat'].isin(disamb[disamb['label not strict']==1]['idCat'].unique())]

In [ ]:
#keep on the necessary fields
positives = augmentedDF[['idCat', 'text', 'ontology', 'classAug']].rename(columns={'text':'sentence'})

#create label
positives['label'] = np.where(positives['classAug']< 4, 1, 0)

In [ ]:
#generate class 0 (original) for the original sentences when labelled positive
#generate class 7 (original contradiction) when labeled negative
disamb['classAug'] = [0 if i == 1 else 7 for i in disamb['label']]

#add sentences labeled as negatives in the human disambiguation component as random negations
negatives_and_originals = pd.merge(disamb[['idCat', 'sentence', 'category', 'classAug', 'label']], o, left_on = 'category', right_index = True).drop('category', axis=1)

#re-order columsn to match positives
negatives_and_originals = negatives_and_originals[['idCat', 'sentence', 'ontology', 'classAug', 'label']]


In [ ]:
# concatenate positive and negative examples
finalDF = pd.concat([positives, negatives_and_originals], axis=0)

In [ ]:
# delete originals
finalDF = finalDF[(finalDF['classAug']!=0) & (finalDF['classAug']!=7)]

In [ ]:
# save final dataset
finalDF.to_excel('/content/drive/MyDrive/DCAST/Financial/Inputs/7.finalDF_Augs_24Oct23.xlsx')

### **AWCL Component**

In [ ]:
# install package
!pip install sentence_transformers --quiet

In [ ]:
# import package
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

**Loss function**

In [ ]:
from typing import Union, List


class InputExampleAWCL:
    """
    Structure for one input example with texts, the label and a unique id
    """
    def __init__(self, guid: str = '', texts: List[str] = None, classAug: Union[int, float] = 0, label: Union[int, float] = 0):
        """
        Creates one InputExample with the given texts, guid and label
        :param guid
            id for the example
        :param texts
            the texts for the example.
        :param label
            the label for the example
        """
        self.guid = guid
        self.texts = texts
        self.classAug = classAug
        self.label = label

    def __str__(self):
        return "<InputExample> label: {}, texts: {}".format(str(self.label), "; ".join(self.texts))

In [ ]:
#I had to chenge the sentence tranformer model so it accepts the class of positive and negatives as inputs

import json
import logging
import os
import shutil
import stat
from collections import OrderedDict
from typing import List, Dict, Tuple, Iterable, Type, Union, Callable, Optional
import requests
import numpy as np
from numpy import ndarray
import transformers
from huggingface_hub import HfApi, HfFolder, Repository, hf_hub_url, cached_download
import torch
from torch import nn, Tensor, device
from torch.optim import Optimizer
from torch.utils.data import DataLoader
import torch.multiprocessing as mp
from tqdm.autonotebook import trange
import math
import queue
import tempfile
from distutils.dir_util import copy_tree

from sentence_transformers import __MODEL_HUB_ORGANIZATION__
from sentence_transformers.evaluation import SentenceEvaluator
from sentence_transformers.util import import_from_string, batch_to_device, fullname, snapshot_download
from sentence_transformers.models import Transformer, Pooling, Dense
from sentence_transformers.model_card_templates import ModelCardTemplate
from sentence_transformers import __version__

logger = logging.getLogger(__name__)

class SentenceTransformerAWCL(nn.Sequential):
    """
    Loads or create a SentenceTransformer model, that can be used to map sentences / text to embeddings.
    :param model_name_or_path: If it is a filepath on disc, it loads the model from that path. If it is not a path, it first tries to download a pre-trained SentenceTransformer model. If that fails, tries to construct a model from Huggingface models repository with that name.
    :param modules: This parameter can be used to create custom SentenceTransformer models from scratch.
    :param device: Device (like 'cuda' / 'cpu') that should be used for computation. If None, checks if a GPU can be used.
    :param cache_folder: Path to store models. Can be also set by SENTENCE_TRANSFORMERS_HOME enviroment variable.
    :param use_auth_token: HuggingFace authentication token to download private models.
    """
    def __init__(self, model_name_or_path: Optional[str] = None,
                 modules: Optional[Iterable[nn.Module]] = None,
                 device: Optional[str] = None,
                 cache_folder: Optional[str] = None,
                 use_auth_token: Union[bool, str, None] = None
                 ):
        self._model_card_vars = {}
        self._model_card_text = None
        self._model_config = {}

        if cache_folder is None:
            cache_folder = os.getenv('SENTENCE_TRANSFORMERS_HOME')
            if cache_folder is None:
                try:
                    from torch.hub import _get_torch_home

                    torch_cache_home = _get_torch_home()
                except ImportError:
                    torch_cache_home = os.path.expanduser(os.getenv('TORCH_HOME', os.path.join(os.getenv('XDG_CACHE_HOME', '~/.cache'), 'torch')))

                cache_folder = os.path.join(torch_cache_home, 'sentence_transformers')

        if model_name_or_path is not None and model_name_or_path != "":
            logger.info("Load pretrained SentenceTransformer: {}".format(model_name_or_path))

            #Old models that don't belong to any organization
            basic_transformer_models = ['albert-base-v1', 'albert-base-v2', 'albert-large-v1', 'albert-large-v2', 'albert-xlarge-v1', 'albert-xlarge-v2', 'albert-xxlarge-v1', 'albert-xxlarge-v2', 'bert-base-cased-finetuned-mrpc', 'bert-base-cased', 'bert-base-chinese', 'bert-base-german-cased', 'bert-base-german-dbmdz-cased', 'bert-base-german-dbmdz-uncased', 'bert-base-multilingual-cased', 'bert-base-multilingual-uncased', 'bert-base-uncased', 'bert-large-cased-whole-word-masking-finetuned-squad', 'bert-large-cased-whole-word-masking', 'bert-large-cased', 'bert-large-uncased-whole-word-masking-finetuned-squad', 'bert-large-uncased-whole-word-masking', 'bert-large-uncased', 'camembert-base', 'ctrl', 'distilbert-base-cased-distilled-squad', 'distilbert-base-cased', 'distilbert-base-german-cased', 'distilbert-base-multilingual-cased', 'distilbert-base-uncased-distilled-squad', 'distilbert-base-uncased-finetuned-sst-2-english', 'distilbert-base-uncased', 'distilgpt2', 'distilroberta-base', 'gpt2-large', 'gpt2-medium', 'gpt2-xl', 'gpt2', 'openai-gpt', 'roberta-base-openai-detector', 'roberta-base', 'roberta-large-mnli', 'roberta-large-openai-detector', 'roberta-large', 't5-11b', 't5-3b', 't5-base', 't5-large', 't5-small', 'transfo-xl-wt103', 'xlm-clm-ende-1024', 'xlm-clm-enfr-1024', 'xlm-mlm-100-1280', 'xlm-mlm-17-1280', 'xlm-mlm-en-2048', 'xlm-mlm-ende-1024', 'xlm-mlm-enfr-1024', 'xlm-mlm-enro-1024', 'xlm-mlm-tlm-xnli15-1024', 'xlm-mlm-xnli15-1024', 'xlm-roberta-base', 'xlm-roberta-large-finetuned-conll02-dutch', 'xlm-roberta-large-finetuned-conll02-spanish', 'xlm-roberta-large-finetuned-conll03-english', 'xlm-roberta-large-finetuned-conll03-german', 'xlm-roberta-large', 'xlnet-base-cased', 'xlnet-large-cased']

            if os.path.exists(model_name_or_path):
                #Load from path
                model_path = model_name_or_path
            else:
                #Not a path, load from hub
                if '\\' in model_name_or_path or model_name_or_path.count('/') > 1:
                    raise ValueError("Path {} not found".format(model_name_or_path))

                if '/' not in model_name_or_path and model_name_or_path.lower() not in basic_transformer_models:
                    # A model from sentence-transformers
                    model_name_or_path = __MODEL_HUB_ORGANIZATION__ + "/" + model_name_or_path

                model_path = os.path.join(cache_folder, model_name_or_path.replace("/", "_"))

                if not os.path.exists(os.path.join(model_path, 'modules.json')):
                    # Download from hub with caching
                    snapshot_download(model_name_or_path,
                                        cache_dir=cache_folder,
                                        library_name='sentence-transformers',
                                        library_version=__version__,
                                        ignore_files=['flax_model.msgpack', 'rust_model.ot', 'tf_model.h5'],
                                        use_auth_token=use_auth_token)

            if os.path.exists(os.path.join(model_path, 'modules.json')):    #Load as SentenceTransformer model
                modules = self._load_sbert_model(model_path)
            else:   #Load with AutoModel
                modules = self._load_auto_model(model_path)

        if modules is not None and not isinstance(modules, OrderedDict):
            modules = OrderedDict([(str(idx), module) for idx, module in enumerate(modules)])

        super().__init__(modules)
        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
            logger.info("Use pytorch device: {}".format(device))

        self._target_device = torch.device(device)


    def encode(self, sentences: Union[str, List[str]],
               batch_size: int = 32,
               show_progress_bar: bool = None,
               output_value: str = 'sentence_embedding',
               convert_to_numpy: bool = True,
               convert_to_tensor: bool = False,
               device: str = None,
               normalize_embeddings: bool = False) -> Union[List[Tensor], ndarray, Tensor]:
        """
        Computes sentence embeddings
        :param sentences: the sentences to embed
        :param batch_size: the batch size used for the computation
        :param show_progress_bar: Output a progress bar when encode sentences
        :param output_value:  Default sentence_embedding, to get sentence embeddings. Can be set to token_embeddings to get wordpiece token embeddings. Set to None, to get all output values
        :param convert_to_numpy: If true, the output is a list of numpy vectors. Else, it is a list of pytorch tensors.
        :param convert_to_tensor: If true, you get one large tensor as return. Overwrites any setting from convert_to_numpy
        :param device: Which torch.device to use for the computation
        :param normalize_embeddings: If set to true, returned vectors will have length 1. In that case, the faster dot-product (util.dot_score) instead of cosine similarity can be used.
        :return:
           By default, a list of tensors is returned. If convert_to_tensor, a stacked tensor is returned. If convert_to_numpy, a numpy matrix is returned.
        """
        self.eval()
        if show_progress_bar is None:
            show_progress_bar = (logger.getEffectiveLevel()==logging.INFO or logger.getEffectiveLevel()==logging.DEBUG)

        if convert_to_tensor:
            convert_to_numpy = False

        if output_value != 'sentence_embedding':
            convert_to_tensor = False
            convert_to_numpy = False

        input_was_string = False
        if isinstance(sentences, str) or not hasattr(sentences, '__len__'): #Cast an individual sentence to a list with length 1
            sentences = [sentences]
            input_was_string = True

        if device is None:
            device = self._target_device

        self.to(device)

        all_embeddings = []
        length_sorted_idx = np.argsort([-self._text_length(sen) for sen in sentences])
        sentences_sorted = [sentences[idx] for idx in length_sorted_idx]

        for start_index in trange(0, len(sentences), batch_size, desc="Batches", disable=not show_progress_bar):
            sentences_batch = sentences_sorted[start_index:start_index+batch_size]
            features = self.tokenize(sentences_batch)
            features = batch_to_device(features, device)

            with torch.no_grad():
                out_features = self.forward(features)

                if output_value == 'token_embeddings':
                    embeddings = []
                    for token_emb, attention in zip(out_features[output_value], out_features['attention_mask']):
                        last_mask_id = len(attention)-1
                        while last_mask_id > 0 and attention[last_mask_id].item() == 0:
                            last_mask_id -= 1

                        embeddings.append(token_emb[0:last_mask_id+1])
                elif output_value is None:  #Return all outputs
                    embeddings = []
                    for sent_idx in range(len(out_features['sentence_embedding'])):
                        row =  {name: out_features[name][sent_idx] for name in out_features}
                        embeddings.append(row)
                else:   #Sentence embeddings
                    embeddings = out_features[output_value]
                    embeddings = embeddings.detach()
                    if normalize_embeddings:
                        embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)

                    # fixes for #522 and #487 to avoid oom problems on gpu with large datasets
                    if convert_to_numpy:
                        embeddings = embeddings.cpu()

                all_embeddings.extend(embeddings)

        all_embeddings = [all_embeddings[idx] for idx in np.argsort(length_sorted_idx)]

        if convert_to_tensor:
            all_embeddings = torch.stack(all_embeddings)
        elif convert_to_numpy:
            all_embeddings = np.asarray([emb.numpy() for emb in all_embeddings])

        if input_was_string:
            all_embeddings = all_embeddings[0]

        return all_embeddings


    def start_multi_process_pool(self, target_devices: List[str] = None):
        """
        Starts multi process to process the encoding with several, independent processes.
        This method is recommended if you want to encode on multiple GPUs. It is advised
        to start only one process per GPU. This method works together with encode_multi_process
        :param target_devices: PyTorch target devices, e.g. cuda:0, cuda:1... If None, all available CUDA devices will be used
        :return: Returns a dict with the target processes, an input queue and and output queue.
        """
        if target_devices is None:
            if torch.cuda.is_available():
                target_devices = ['cuda:{}'.format(i) for i in range(torch.cuda.device_count())]
            else:
                logger.info("CUDA is not available. Start 4 CPU worker")
                target_devices = ['cpu']*4

        logger.info("Start multi-process pool on devices: {}".format(', '.join(map(str, target_devices))))

        ctx = mp.get_context('spawn')
        input_queue = ctx.Queue()
        output_queue = ctx.Queue()
        processes = []

        for cuda_id in target_devices:
            p = ctx.Process(target=SentenceTransformer._encode_multi_process_worker, args=(cuda_id, self, input_queue, output_queue), daemon=True)
            p.start()
            processes.append(p)

        return {'input': input_queue, 'output': output_queue, 'processes': processes}

    @staticmethod
    def stop_multi_process_pool(pool):
        """
        Stops all processes started with start_multi_process_pool
        """
        for p in pool['processes']:
            p.terminate()

        for p in pool['processes']:
            p.join()
            p.close()

        pool['input'].close()
        pool['output'].close()

    def encode_multi_process(self, sentences: List[str], pool: Dict[str, object], batch_size: int = 32, chunk_size: int = None):
        """
        This method allows to run encode() on multiple GPUs. The sentences are chunked into smaller packages
        and sent to individual processes, which encode these on the different GPUs. This method is only suitable
        for encoding large sets of sentences
        :param sentences: List of sentences
        :param pool: A pool of workers started with SentenceTransformer.start_multi_process_pool
        :param batch_size: Encode sentences with batch size
        :param chunk_size: Sentences are chunked and sent to the individual processes. If none, it determine a sensible size.
        :return: Numpy matrix with all embeddings
        """

        if chunk_size is None:
            chunk_size = min(math.ceil(len(sentences) / len(pool["processes"]) / 10), 5000)

        logger.debug(f"Chunk data into {math.ceil(len(sentences) / chunk_size)} packages of size {chunk_size}")

        input_queue = pool['input']
        last_chunk_id = 0
        chunk = []

        for sentence in sentences:
            chunk.append(sentence)
            if len(chunk) >= chunk_size:
                input_queue.put([last_chunk_id, batch_size, chunk])
                last_chunk_id += 1
                chunk = []

        if len(chunk) > 0:
            input_queue.put([last_chunk_id, batch_size, chunk])
            last_chunk_id += 1

        output_queue = pool['output']
        results_list = sorted([output_queue.get() for _ in range(last_chunk_id)], key=lambda x: x[0])
        embeddings = np.concatenate([result[1] for result in results_list])
        return embeddings

    @staticmethod
    def _encode_multi_process_worker(target_device: str, model, input_queue, results_queue):
        """
        Internal working process to encode sentences in multi-process setup
        """
        while True:
            try:
                id, batch_size, sentences = input_queue.get()
                embeddings = model.encode(sentences, device=target_device,  show_progress_bar=False, convert_to_numpy=True, batch_size=batch_size)
                results_queue.put([id, embeddings])
            except queue.Empty:
                break


    def get_max_seq_length(self):
        """
        Returns the maximal sequence length for input the model accepts. Longer inputs will be truncated
        """
        if hasattr(self._first_module(), 'max_seq_length'):
            return self._first_module().max_seq_length

        return None

    def tokenize(self, texts: Union[List[str], List[Dict], List[Tuple[str, str]]]):
        """
        Tokenizes the texts
        """
        return self._first_module().tokenize(texts)

    def get_sentence_features(self, *features):
        return self._first_module().get_sentence_features(*features)

    def get_sentence_embedding_dimension(self):
        for mod in reversed(self._modules.values()):
            sent_embedding_dim_method = getattr(mod, "get_sentence_embedding_dimension", None)
            if callable(sent_embedding_dim_method):
                return sent_embedding_dim_method()
        return None

    def _first_module(self):
        """Returns the first module of this sequential embedder"""
        return self._modules[next(iter(self._modules))]

    def _last_module(self):
        """Returns the last module of this sequential embedder"""
        return self._modules[next(reversed(self._modules))]

    def save(self, path: str, model_name: Optional[str] = None, create_model_card: bool = True, train_datasets: Optional[List[str]] = None):
        """
        Saves all elements for this seq. sentence embedder into different sub-folders
        :param path: Path on disc
        :param model_name: Optional model name
        :param create_model_card: If True, create a README.md with basic information about this model
        :param train_datasets: Optional list with the names of the datasets used to to train the model
        """
        if path is None:
            return

        os.makedirs(path, exist_ok=True)

        logger.info("Save model to {}".format(path))
        modules_config = []

        #Save some model info
        if '__version__' not in self._model_config:
            self._model_config['__version__'] = {
                    'sentence_transformers': __version__,
                    'transformers': transformers.__version__,
                    'pytorch': torch.__version__,
                }

        with open(os.path.join(path, 'config_sentence_transformers.json'), 'w') as fOut:
            json.dump(self._model_config, fOut, indent=2)

        #Save modules
        for idx, name in enumerate(self._modules):
            module = self._modules[name]
            if idx == 0 and isinstance(module, Transformer):    #Save transformer model in the main folder
                model_path = path + "/"
            else:
                model_path = os.path.join(path, str(idx)+"_"+type(module).__name__)

            os.makedirs(model_path, exist_ok=True)
            module.save(model_path)
            modules_config.append({'idx': idx, 'name': name, 'path': os.path.basename(model_path), 'type': type(module).__module__})

        with open(os.path.join(path, 'modules.json'), 'w') as fOut:
            json.dump(modules_config, fOut, indent=2)

        # Create model card
        if create_model_card:
            self._create_model_card(path, model_name, train_datasets)

    def _create_model_card(self, path: str, model_name: Optional[str] = None, train_datasets: Optional[List[str]] = None):
        """
        Create an automatic model and stores it in path
        """
        if self._model_card_text is not None and len(self._model_card_text) > 0:
            model_card = self._model_card_text
        else:
            tags = ModelCardTemplate.__TAGS__.copy()
            model_card = ModelCardTemplate.__MODEL_CARD__

            if len(self._modules) == 2 and isinstance(self._first_module(), Transformer) and isinstance(self._last_module(), Pooling) and self._last_module().get_pooling_mode_str() in ['cls', 'max', 'mean']:
                pooling_module = self._last_module()
                pooling_mode = pooling_module.get_pooling_mode_str()
                model_card = model_card.replace("{USAGE_TRANSFORMERS_SECTION}", ModelCardTemplate.__USAGE_TRANSFORMERS__)
                pooling_fct_name, pooling_fct = ModelCardTemplate.model_card_get_pooling_function(pooling_mode)
                model_card = model_card.replace("{POOLING_FUNCTION}", pooling_fct).replace("{POOLING_FUNCTION_NAME}", pooling_fct_name).replace("{POOLING_MODE}", pooling_mode)
                tags.append('transformers')

            # Print full model
            model_card = model_card.replace("{FULL_MODEL_STR}", str(self))

            # Add tags
            model_card = model_card.replace("{TAGS}", "\n".join(["- "+t for t in tags]))

            datasets_str = ""
            if train_datasets is not None:
                datasets_str = "datasets:\n"+"\n".join(["- " + d for d in train_datasets])
            model_card = model_card.replace("{DATASETS}", datasets_str)

            # Add dim info
            self._model_card_vars["{NUM_DIMENSIONS}"] = self.get_sentence_embedding_dimension()

            # Replace vars we created while using the model
            for name, value in self._model_card_vars.items():
                model_card = model_card.replace(name, str(value))

            # Replace remaining vars with default values
            for name, value in ModelCardTemplate.__DEFAULT_VARS__.items():
                model_card = model_card.replace(name, str(value))

        if model_name is not None:
            model_card = model_card.replace("{MODEL_NAME}", model_name.strip())

        with open(os.path.join(path, "README.md"), "w", encoding='utf8') as fOut:
            fOut.write(model_card.strip())

    def save_to_hub(self,
                    repo_name: str,
                    organization: Optional[str] = None,
                    private: Optional[bool] = None,
                    commit_message: str = "Add new SentenceTransformer model.",
                    local_model_path: Optional[str] = None,
                    exist_ok: bool = False,
                    replace_model_card: bool = False,
                    train_datasets: Optional[List[str]] = None):
        """
        Uploads all elements of this Sentence Transformer to a new HuggingFace Hub repository.
        :param repo_name: Repository name for your model in the Hub.
        :param organization:  Organization in which you want to push your model or tokenizer (you must be a member of this organization).
        :param private: Set to true, for hosting a prive model
        :param commit_message: Message to commit while pushing.
        :param local_model_path: Path of the model locally. If set, this file path will be uploaded. Otherwise, the current model will be uploaded
        :param exist_ok: If true, saving to an existing repository is OK. If false, saving only to a new repository is possible
        :param replace_model_card: If true, replace an existing model card in the hub with the automatically created model card
        :param train_datasets: Datasets used to train the model. If set, the datasets will be added to the model card in the Hub.
        :return: The url of the commit of your model in the given repository.
        """
        token = HfFolder.get_token()
        if token is None:
            raise ValueError("You must login to the Hugging Face hub on this computer by typing `transformers-cli login`.")

        if '/' in repo_name:
            splits = repo_name.split('/', maxsplit=1)
            if organization is None or organization == splits[0]:
                organization = splits[0]
                repo_name = splits[1]
            else:
                raise ValueError("You passed and invalid repository name: {}.".format(repo_name))

        endpoint = "https://huggingface.co"
        repo_id = repo_name
        if organization:
          repo_id = f"{organization}/{repo_id}"
        repo_url = HfApi(endpoint=endpoint).create_repo(
                repo_id=repo_id,
                token=token,
                private=private,
                repo_type=None,
                exist_ok=exist_ok,
            )
        full_model_name = repo_url[len(endpoint)+1:].strip("/")

        with tempfile.TemporaryDirectory() as tmp_dir:
            # First create the repo (and clone its content if it's nonempty).
            logger.info("Create repository and clone it if it exists")
            repo = Repository(tmp_dir, clone_from=repo_url)

            # If user provides local files, copy them.
            if local_model_path:
                copy_tree(local_model_path, tmp_dir)
            else:  # Else, save model directly into local repo.
                create_model_card = replace_model_card or not os.path.exists(os.path.join(tmp_dir, 'README.md'))
                self.save(tmp_dir, model_name=full_model_name, create_model_card=create_model_card, train_datasets=train_datasets)

            #Find files larger 5M and track with git-lfs
            large_files = []
            for root, dirs, files in os.walk(tmp_dir):
                for filename in files:
                    file_path = os.path.join(root, filename)
                    rel_path = os.path.relpath(file_path, tmp_dir)

                    if os.path.getsize(file_path) > (5 * 1024 * 1024):
                        large_files.append(rel_path)

            if len(large_files) > 0:
                logger.info("Track files with git lfs: {}".format(", ".join(large_files)))
                repo.lfs_track(large_files)

            logger.info("Push model to the hub. This might take a while")
            push_return = repo.push_to_hub(commit_message=commit_message)

            def on_rm_error(func, path, exc_info):
                # path contains the path of the file that couldn't be removed
                # let's just assume that it's read-only and unlink it.
                try:
                    os.chmod(path, stat.S_IWRITE)
                    os.unlink(path)
                except:
                    pass

            # Remove .git folder. On Windows, the .git folder might be read-only and cannot be deleted
            # Hence, try to set write permissions on error
            try:
                for f in os.listdir(tmp_dir):
                    shutil.rmtree(os.path.join(tmp_dir, f), onerror=on_rm_error)
            except Exception as e:
                logger.warning("Error when deleting temp folder: {}".format(str(e)))
                pass

        return push_return

    def smart_batching_collate(self, batch):
        """
        Transforms a batch from a SmartBatchingDataset to a batch of tensors for the model
        Here, batch is a list of tuples: [(tokens, label), ...]
        :param batch:
            a batch from a SmartBatchingDataset
        :return:
            a batch of tensors for the model
        """
        num_texts = len(batch[0].texts)
        texts = [[] for _ in range(num_texts)]
        labels = []
        classAugs = []

        for example in batch:
            for idx, text in enumerate(example.texts):
                texts[idx].append(text)

            labels.append(example.label)
            classAugs.append(example.classAug)

        labels = torch.tensor(labels)
        classAugs = torch.tensor(classAugs)

        sentence_features = []
        for idx in range(num_texts):
            tokenized = self.tokenize(texts[idx])
            sentence_features.append(tokenized)

        return sentence_features, classAugs, labels

    def _text_length(self, text: Union[List[int], List[List[int]]]):
        """
        Help function to get the length for the input text. Text can be either
        a list of ints (which means a single text as input), or a tuple of list of ints
        (representing several text inputs to the model).
        """

        if isinstance(text, dict):              #{key: value} case
            return len(next(iter(text.values())))
        elif not hasattr(text, '__len__'):      #Object has no len() method
            return 1
        elif len(text) == 0 or isinstance(text[0], int):    #Empty string or list of ints
            return len(text)
        else:
            return sum([len(t) for t in text])      #Sum of length of individual strings

    def fit(self,
            train_objectives: Iterable[Tuple[DataLoader, nn.Module]],
            evaluator: SentenceEvaluator = None,
            epochs: int = 1,
            steps_per_epoch = None,
            scheduler: str = 'WarmupLinear',
            warmup_steps: int = 10000,
            optimizer_class: Type[Optimizer] = torch.optim.AdamW,
            optimizer_params : Dict[str, object]= {'lr': 2e-5},
            weight_decay: float = 0.01,
            evaluation_steps: int = 0,
            output_path: str = None,
            save_best_model: bool = True,
            max_grad_norm: float = 1,
            use_amp: bool = False,
            callback: Callable[[float, int, int], None] = None,
            show_progress_bar: bool = True,
            checkpoint_path: str = None,
            checkpoint_save_steps: int = 500,
            checkpoint_save_total_limit: int = 0
            ):
        """
        Train the model with the given training objective
        Each training objective is sampled in turn for one batch.
        We sample only as many batches from each objective as there are in the smallest one
        to make sure of equal training with each dataset.
        :param train_objectives: Tuples of (DataLoader, LossFunction). Pass more than one for multi-task learning
        :param evaluator: An evaluator (sentence_transformers.evaluation) evaluates the model performance during training on held-out dev data. It is used to determine the best model that is saved to disc.
        :param epochs: Number of epochs for training
        :param steps_per_epoch: Number of training steps per epoch. If set to None (default), one epoch is equal the DataLoader size from train_objectives.
        :param scheduler: Learning rate scheduler. Available schedulers: constantlr, warmupconstant, warmuplinear, warmupcosine, warmupcosinewithhardrestarts
        :param warmup_steps: Behavior depends on the scheduler. For WarmupLinear (default), the learning rate is increased from o up to the maximal learning rate. After these many training steps, the learning rate is decreased linearly back to zero.
        :param optimizer_class: Optimizer
        :param optimizer_params: Optimizer parameters
        :param weight_decay: Weight decay for model parameters
        :param evaluation_steps: If > 0, evaluate the model using evaluator after each number of training steps
        :param output_path: Storage path for the model and evaluation files
        :param save_best_model: If true, the best model (according to evaluator) is stored at output_path
        :param max_grad_norm: Used for gradient normalization.
        :param use_amp: Use Automatic Mixed Precision (AMP). Only for Pytorch >= 1.6.0
        :param callback: Callback function that is invoked after each evaluation.
                It must accept the following three parameters in this order:
                `score`, `epoch`, `steps`
        :param show_progress_bar: If True, output a tqdm progress bar
        :param checkpoint_path: Folder to save checkpoints during training
        :param checkpoint_save_steps: Will save a checkpoint after so many steps
        :param checkpoint_save_total_limit: Total number of checkpoints to store
        """

        ##Add info to model card
        #info_loss_functions = "\n".join(["- {} with {} training examples".format(str(loss), len(dataloader)) for dataloader, loss in train_objectives])
        info_loss_functions =  []
        for dataloader, loss in train_objectives:
            info_loss_functions.extend(ModelCardTemplate.get_train_objective_info(dataloader, loss))
        info_loss_functions = "\n\n".join([text for text in info_loss_functions])

        info_fit_parameters = json.dumps({"evaluator": fullname(evaluator), "epochs": epochs, "steps_per_epoch": steps_per_epoch, "scheduler": scheduler, "warmup_steps": warmup_steps, "optimizer_class": str(optimizer_class),  "optimizer_params": optimizer_params, "weight_decay": weight_decay, "evaluation_steps": evaluation_steps, "max_grad_norm": max_grad_norm }, indent=4, sort_keys=True)
        self._model_card_text = None
        self._model_card_vars['{TRAINING_SECTION}'] = ModelCardTemplate.__TRAINING_SECTION__.replace("{LOSS_FUNCTIONS}", info_loss_functions).replace("{FIT_PARAMETERS}", info_fit_parameters)

        if use_amp:
            from torch.cuda.amp import autocast
            scaler = torch.cuda.amp.GradScaler()

        self.to(self._target_device)

        dataloaders = [dataloader for dataloader, _ in train_objectives]

        # Use smart batching
        for dataloader in dataloaders:
            dataloader.collate_fn = self.smart_batching_collate

        loss_models = [loss for _, loss in train_objectives]
        for loss_model in loss_models:
            loss_model.to(self._target_device)

        self.best_score = -9999999

        if steps_per_epoch is None or steps_per_epoch == 0:
            steps_per_epoch = min([len(dataloader) for dataloader in dataloaders])

        num_train_steps = int(steps_per_epoch * epochs)

        # Prepare optimizers
        optimizers = []
        schedulers = []
        for loss_model in loss_models:
            param_optimizer = list(loss_model.named_parameters())

            no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
            optimizer_grouped_parameters = [
                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
            ]

            optimizer = optimizer_class(optimizer_grouped_parameters, **optimizer_params)
            scheduler_obj = self._get_scheduler(optimizer, scheduler=scheduler, warmup_steps=warmup_steps, t_total=num_train_steps)

            optimizers.append(optimizer)
            schedulers.append(scheduler_obj)

        global_step = 0
        data_iterators = [iter(dataloader) for dataloader in dataloaders]

        num_train_objectives = len(train_objectives)

        skip_scheduler = False
        for epoch in trange(epochs, desc="Epoch", disable=not show_progress_bar):
            training_steps = 0

            for loss_model in loss_models:
                loss_model.zero_grad()
                loss_model.train()

            for _ in trange(steps_per_epoch, desc="Iteration", smoothing=0.05, disable=not show_progress_bar):
                for train_idx in range(num_train_objectives):
                    loss_model = loss_models[train_idx]
                    optimizer = optimizers[train_idx]
                    scheduler = schedulers[train_idx]
                    data_iterator = data_iterators[train_idx]

                    try:
                        data = next(data_iterator)
                    except StopIteration:
                        data_iterator = iter(dataloaders[train_idx])
                        data_iterators[train_idx] = data_iterator
                        data = next(data_iterator)

                    features, classAugs, labels = data

                    classAugs = classAugs.to(self._target_device)
                    labels = labels.to(self._target_device)
                    features = list(map(lambda batch: batch_to_device(batch, self._target_device), features))

                    if use_amp:
                        with autocast():
                            loss_value = loss_model(features, classAugs, labels)

                        scale_before_step = scaler.get_scale()
                        scaler.scale(loss_value).backward()
                        scaler.unscale_(optimizer)
                        torch.nn.utils.clip_grad_norm_(loss_model.parameters(), max_grad_norm)
                        scaler.step(optimizer)
                        scaler.update()

                        skip_scheduler = scaler.get_scale() != scale_before_step
                    else:
                        loss_value = loss_model(features, classAugs, labels)
                        loss_value.backward()
                        torch.nn.utils.clip_grad_norm_(loss_model.parameters(), max_grad_norm)
                        optimizer.step()

                    optimizer.zero_grad()

                    if not skip_scheduler:
                        scheduler.step()

                training_steps += 1
                global_step += 1

                if evaluation_steps > 0 and training_steps % evaluation_steps == 0:
                    self._eval_during_training(evaluator, output_path, save_best_model, epoch, training_steps, callback)

                    for loss_model in loss_models:
                        loss_model.zero_grad()
                        loss_model.train()

                if checkpoint_path is not None and checkpoint_save_steps is not None and checkpoint_save_steps > 0 and global_step % checkpoint_save_steps == 0:
                    self._save_checkpoint(checkpoint_path, checkpoint_save_total_limit, global_step)

            self._eval_during_training(evaluator, output_path, save_best_model, epoch, -1, callback)

        if evaluator is None and output_path is not None:   #No evaluator, but output path: save final model version
            self.save(output_path)

        if checkpoint_path is not None:
            self._save_checkpoint(checkpoint_path, checkpoint_save_total_limit, global_step)


    def evaluate(self, evaluator: SentenceEvaluator, output_path: str = None):
        """
        Evaluate the model
        :param evaluator:
            the evaluator
        :param output_path:
            the evaluator can write the results to this path
        """
        if output_path is not None:
            os.makedirs(output_path, exist_ok=True)
        return evaluator(self, output_path)

    def _eval_during_training(self, evaluator, output_path, save_best_model, epoch, steps, callback):
        """Runs evaluation during the training"""
        eval_path = output_path
        if output_path is not None:
            os.makedirs(output_path, exist_ok=True)
            eval_path = os.path.join(output_path, "eval")
            os.makedirs(eval_path, exist_ok=True)

        if evaluator is not None:
            score = evaluator(self, output_path=eval_path, epoch=epoch, steps=steps)
            if callback is not None:
                callback(score, epoch, steps)
            if score > self.best_score:
                self.best_score = score
                if save_best_model:
                    self.save(output_path)

    def _save_checkpoint(self, checkpoint_path, checkpoint_save_total_limit, step):
        # Store new checkpoint
        self.save(os.path.join(checkpoint_path, str(step)))

        # Delete old checkpoints
        if checkpoint_save_total_limit is not None and checkpoint_save_total_limit > 0:
            old_checkpoints = []
            for subdir in os.listdir(checkpoint_path):
                if subdir.isdigit():
                    old_checkpoints.append({'step': int(subdir), 'path': os.path.join(checkpoint_path, subdir)})

            if len(old_checkpoints) > checkpoint_save_total_limit:
                old_checkpoints = sorted(old_checkpoints, key=lambda x: x['step'])
                shutil.rmtree(old_checkpoints[0]['path'])

    def _load_auto_model(self, model_name_or_path):
        """
        Creates a simple Transformer + Mean Pooling model and returns the modules
        """
        logger.warning("No sentence-transformers model found with name {}. Creating a new one with MEAN pooling.".format(model_name_or_path))
        transformer_model = Transformer(model_name_or_path)
        pooling_model = Pooling(transformer_model.get_word_embedding_dimension(), 'mean')
        return [transformer_model, pooling_model]

    def _load_sbert_model(self, model_path):
        """
        Loads a full sentence-transformers model
        """
        # Check if the config_sentence_transformers.json file exists (exists since v2 of the framework)
        config_sentence_transformers_json_path = os.path.join(model_path, 'config_sentence_transformers.json')
        if os.path.exists(config_sentence_transformers_json_path):
            with open(config_sentence_transformers_json_path) as fIn:
                self._model_config = json.load(fIn)

            if '__version__' in self._model_config and 'sentence_transformers' in self._model_config['__version__'] and self._model_config['__version__']['sentence_transformers'] > __version__:
                logger.warning("You try to use a model that was created with version {}, however, your version is {}. This might cause unexpected behavior or errors. In that case, try to update to the latest version.\n\n\n".format(self._model_config['__version__']['sentence_transformers'], __version__))

        # Check if a readme exists
        model_card_path = os.path.join(model_path, 'README.md')
        if os.path.exists(model_card_path):
            try:
                with open(model_card_path, encoding='utf8') as fIn:
                    self._model_card_text = fIn.read()
            except:
                pass

        # Load the modules of sentence transformer
        modules_json_path = os.path.join(model_path, 'modules.json')
        with open(modules_json_path) as fIn:
            modules_config = json.load(fIn)

        modules = OrderedDict()
        for module_config in modules_config:
            module_class = import_from_string(module_config['type'])
            module = module_class.load(os.path.join(model_path, module_config['path']))
            modules[module_config['name']] = module

        return modules

    @staticmethod
    def load(input_path):
        return SentenceTransformer(input_path)

    @staticmethod
    def _get_scheduler(optimizer, scheduler: str, warmup_steps: int, t_total: int):
        """
        Returns the correct learning rate scheduler. Available scheduler: constantlr, warmupconstant, warmuplinear, warmupcosine, warmupcosinewithhardrestarts
        """
        scheduler = scheduler.lower()
        if scheduler == 'constantlr':
            return transformers.get_constant_schedule(optimizer)
        elif scheduler == 'warmupconstant':
            return transformers.get_constant_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps)
        elif scheduler == 'warmuplinear':
            return transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)
        elif scheduler == 'warmupcosine':
            return transformers.get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)
        elif scheduler == 'warmupcosinewithhardrestarts':
            return transformers.get_cosine_with_hard_restarts_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)
        else:
            raise ValueError("Unknown scheduler {}".format(scheduler))

    @property
    def device(self) -> device:
        """
        Get torch.device from module, assuming that the whole module has one device.
        """
        try:
            return next(self.parameters()).device
        except StopIteration:
            # For nn.DataParallel compatibility in PyTorch 1.5

            def find_tensor_attributes(module: nn.Module) -> List[Tuple[str, Tensor]]:
                tuples = [(k, v) for k, v in module.__dict__.items() if torch.is_tensor(v)]
                return tuples

            gen = self._named_members(get_members_fn=find_tensor_attributes)
            first_tuple = next(gen)
            return first_tuple[1].device

    @property
    def tokenizer(self):
        """
        Property to get the tokenizer that is used by this model
        """
        return self._first_module().tokenizer

    @tokenizer.setter
    def tokenizer(self, value):
        """
        Property to set the tokenizer that should be used by this model
        """
        self._first_module().tokenizer = value

    @property
    def max_seq_length(self):
        """
        Property to get the maximal input sequence length for the model. Longer inputs will be truncated.
        """
        return self._first_module().max_seq_length

    @max_seq_length.setter
    def max_seq_length(self, value):
        """
        Property to set the maximal input sequence length for the model. Longer inputs will be truncated.
        """
        self._first_module().max_seq_length = value



In [ ]:
from enum import Enum
from typing import Iterable, Dict
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, Tensor
from sentence_transformers.SentenceTransformer import SentenceTransformer


class AWCLContrastiveLoss(nn.Module):
    def __init__(self, model: SentenceTransformerAWCL, weights: torch.Tensor, margin: float = 0.5, size_average:bool = True, measure: str = "cosine"):
        super(AWCLContrastiveLoss, self).__init__()
        self.model = model
        self.weights = weights
        self.margin = margin
        self.size_average = size_average
        self.measure = measure

    def forward(self, sentence_features: Iterable[Dict[str, Tensor]], classAug: torch.Tensor, label: torch.Tensor) -> torch.Tensor:

        reps = [self.model(sentence_feature)['sentence_embedding'] for sentence_feature in sentence_features]
        assert len(reps) == 2
        sentence, ontology = reps

        if self.measure == "cosine":
            dot_product = torch.sum(sentence * ontology, dim=1)
            magnitudes = torch.sqrt(torch.sum(sentence.pow(2), dim=1) * torch.sum(ontology.pow(2), dim=1))
            distance = 1 - (dot_product / magnitudes)

        weight = self.weights[classAug]  # look up weight for each class

        losses = 0.5 * (label.float() * distance.pow(2) + (1 - label).float() * F.relu(self.margin - distance).pow(2)) * weight

        return losses.mean() if self.size_average else losses.sum()


**Fine-tunning**

In [ ]:
import pandas as pd
#finalDF = pd.read_excel('/content/drive/MyDrive/DCAST/Financial/Inputs/7.finalDF_24Oct23.xlsx', index_col = 0)
finalDF = pd.read_excel('/content/drive/MyDrive/DCAST/Financial/Inputs/7.finalDF_OLD.xlsx', index_col = 0)

In [ ]:
finalDF.reset_index(inplace=True,drop=True)

In [ ]:
dfSamples = finalDF.reset_index(drop=True)
ind = dfSamples.groupby(['ontology', 'label'], group_keys=False).apply(lambda x: x.sample(frac=0.7, random_state=17)).index
train_df = dfSamples[dfSamples.index.isin(ind)]
dev_df = dfSamples[~dfSamples.index.isin(ind)]

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
import math
#from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util
from sentence_transformers import InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, BinaryClassificationEvaluator
import logging
from datetime import datetime


model_name = 'all-mpnet-base-v2'
train_batch_size = 64
num_epochs = 2
model_save_path = '/content/drive/MyDrive/DCAST/Financial/Outputs/fine_tuned_models/'+model_name+'-' + 'withWeights-V7-' +datetime.now().strftime("%Y-%m-%d_%H-%M")

# Load a pre-trained sentence transformer model
model = SentenceTransformerAWCL(model_name)

train_samples = []
dev_samples = []

for index, row in train_df.iterrows():
        inp_example = InputExampleAWCL(texts=[row.sentence, row.ontology], classAug=row.classAug, label=row.label)
        train_samples.append(inp_example)

for index, row in dev_df.iterrows():
        inp_example_dev = InputExample(texts=[row.sentence, row.ontology], label=row.label)
        dev_samples.append(inp_example_dev)


train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)

# weights is a tensor of shape (8,) containing the weight for each class
# the order is as follows (original positive sentence, core arguments, redudancy, nominalization, lexical, negation, random, original negative sentence ) where first three are positives and the last three are negatives
weights = torch.tensor([0.9,0.9,0.9,1.2,1.1,1.3]).to('cuda')
train_loss = AWCLContrastiveLoss(model=model, weights=weights)

# Development set: Measure correlation between cosine score and gold labels
evaluator = BinaryClassificationEvaluator.from_input_examples(dev_samples, name='fine-tune-dev')

# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up

model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          optimizer_class=torch.optim.AdamW,
          output_path=model_save_path,
          save_best_model=True)


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/25 [00:00<?, ?it/s]

Iteration:   0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
!pip install sentence_transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.9 MB/s eta 0:00:00


In [ ]:
!pip install scikeras --quiet

**Re-encoding training set to define thresholds (mean + 3 std)**

In [ ]:
import pandas as pd
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer, util

In [ ]:
df_train = pickle.load(open('/content/drive/MyDrive/DCAST/Financial/Inputs/Sample3000DF.pickle', "rb"))

In [ ]:
df_train = df_train[['key', 'sentence']].sample(100000, random_state=17)

In [ ]:
#encode test set

model_name = 'all-mpnet-base-v2-withWeights-V7-2023-10-26_18-40'
model_save_path = '/content/drive/MyDrive/DCAST/Financial/Outputs/fine_tuned_models/' + model_name


model = SentenceTransformer(model_save_path)
query_embedding = model.encode(["introduced a new generation of products",
      "extended the product range", "opened new markets",
      "entered a new technology field" ,"improved an existing product quality",
      "improved production flexibility", "reduced production cost",
      "improved yield or reduced material consumption"])

df_train['encoding'] = [model.encode(v) for v in df_train['sentence']]

In [ ]:
df_train = df_train[['key', 'sentence','encoding']].reset_index(drop=True)

In [ ]:
df_train = df_train.join(pd.DataFrame(util.cos_sim(np.array(df_train['encoding'].values.tolist()), query_embedding)))

In [ ]:
pickle.dump(df_train, open('/content/drive/MyDrive/DCAST/Financial/Outputs/saved_thresholds/df_train_26Oct23v1.pickle', 'wb'))

In [ ]:
thresholds_dict = {}
for c in range(0, 8):
  for s in range(0, 4):
    thresholds_dict['tm'+str(s)+str(c)] = df_train[[c]].mean() + s*df_train[[c]].std()


In [ ]:
pickle.dump(thresholds_dict, open('/content/drive/MyDrive/DCAST/Financial/Outputs/saved_thresholds/thresholds_main_model_26Oct23v1.pickle', 'wb'))

In [ ]:
thresholds_dict = pickle.load(open('/content/drive/MyDrive/DCAST/Financial/Outputs/saved_thresholds/thresholds_main_model_26Oct23v1.pickle', 'rb'))

### **Classifier Component**

In [ ]:
# mlp for multi-label classification
from numpy import mean
from numpy import std
import numpy as np
import pandas as pd
import pickle
import os
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import KFold, ShuffleSplit, StratifiedKFold, RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import OneHotEncoder
import ast
from keras.optimizers import SGD
from sentence_transformers import SentenceTransformer, util

In [ ]:
#######################reencode corpus
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util

#model_name = 'all-mpnet-base-v2-withWeights-V7-2023-10-24_14-15'
#model_save_path = '/content/drive/MyDrive/DCAST/Financial/Outputs/fine_tuned_models/all-mpnet-base-v2-withWeights-V7-2023-10-24_14-15'
output_file_name = model_name + '.pickle'
encoded_corpus_path = str(os.path.join("/content/drive/MyDrive/DCAST/Financial/Outputs/enconded_corpus", output_file_name))


In [ ]:
finalDF = pd.read_excel('/content/drive/MyDrive/DCAST/Financial/Inputs/7.finalDF_24Oct23.xlsx', index_col = 0)
finalDF.reset_index(inplace=True, drop=True)

In [ ]:
#encode final df
model = SentenceTransformer(model_save_path)
finalDF['encoding'] = [model.encode(v) for v in finalDF['sentence']]

In [ ]:
# Encode ontology
model = SentenceTransformer(model_save_path)
query_embedding = model.encode(["introduced a new generation of products",
      "extended the product range", "opened new markets",
      "entered a new technology field" ,"improved an existing product quality",
      "improved production flexibility", "reduced production cost",
      "improved yield or reduced material consumption"])

In [ ]:
finalDF = finalDF.join(pd.DataFrame(util.cos_sim(np.array(finalDF['encoding'].values.tolist()), query_embedding)))

In [ ]:
pickle.dump(finalDF, open('/content/drive/MyDrive/DCAST/Financial/Outputs/enconded_corpus/finalDFencoded_24Jan24v1.pickle', 'wb'))

In [ ]:
finalDF = pickle.load(open('/content/drive/MyDrive/DCAST/Financial/Outputs/enconded_corpus/finalDFencoded_24Jan24v1.pickle', 'rb'))

In [ ]:
for i in range(0, 8):
  finalDF['ont'+str(i)] = finalDF.apply(lambda r: mean(query_embedding[i]), axis=1)

In [ ]:
finalDF['encoding'] = [mean(e) for e in finalDF['encoding']]

In [ ]:
#finalDF = finalDF[(finalDF['classAug']==0) | (finalDF['classAug']==7)]

In [ ]:
# load dataset
enc = OneHotEncoder(handle_unknown='ignore')
Z = np.array(finalDF['ontology']).reshape(-1, 1)
enc.fit(Z)
enc.categories_
Z1 = enc.transform(Z)
X_cos_sim = pd.DataFrame(finalDF.iloc[:,6:14]).to_numpy()
X_encoding = pd.DataFrame(finalDF[['encoding']]).to_numpy()
X_ontology = pd.DataFrame(finalDF.iloc[:,15:]).to_numpy()
y = pd.DataFrame(Z1.toarray(), columns=enc.categories_).to_numpy()


In [ ]:
ontology = ['entered a new technology field', 'extended the product range', 'improved an existing product quality', 'improved production flexibility', 'improved yield or reduced material consumption', 'introduced a new generation of products', 'opened new markets', 'reduced production cost']

In [ ]:
# get the model
# from keras.metrics import binary_accuracy
# def get_model(n_inputs, n_outputs):
#   model = Sequential()
#   model.add(Dense(8, input_dim=n_inputs, activation='relu'))
#   model.add(Dense(n_outputs, activation='softmax'))
#   #model.compile(loss='binary_crossentropy', optimizer='adam')
#   model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[binary_accuracy])
#   return model

# # evaluate a model using repeated k-fold cross-validation
# def evaluate_model(X, y):
#   results = list()
#   n_inputs, n_outputs = X.shape[1], y.shape[1]
#   # define evaluation procedure
#   #cv =  ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
#   cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=1)
#   # enumerate folds
#   for train, test in cv.split(X, y):
#     # prepare data
#     X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
#     # define model
#     model = get_model(n_inputs, n_outputs)
#     # fit model
#     model.fit(X_train, y_train, epochs=20, batch_size=100, verbose=0)
#     # make a prediction on the test set
#     yhat = model.predict(X_test)
#     # round probabilities to class labels
#     yhat = yhat.round()
#     #yhat = (yhat > 0.5)
#     # calculate accuracy
#     acc = f1_score(y_test, yhat, average='micro')
#     # store result
#     print('>%.3f' % acc)
#     results.append(acc)
#     return results, model, y[train], yhat


# from keras.metrics import binary_accuracy
# def get_model(n_inputs, n_outputs):
#   model = Sequential()
#   layer1 = Dense(1, input_dim=n_inputs, activation='relu')
#   layer1.trainable=True
#   model.add(layer1)
#   layer2 = Dense(n_outputs, activation='sigmoid')
#   layer2.trainable=False
#   model.add(layer2)
#   model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[binary_accuracy])
#   return model

from keras.metrics import binary_accuracy
def get_model(n_inputs, n_outputs):
  model = Sequential()

  layer1 = Dense(1, input_dim=n_inputs, activation='tanh')
  layer1.trainable=True
  model.add(layer1)

  #add cos_sim here
  layer2 = Dense(1, input_dim=n_inputs, activation='tanh')
  layer1.trainable=True
  model.add(layer1)

  #add attention layer (learn to pay more attention to the cosine similarity)

  layer3 = Dense(n_outputs, activation='sigmoid')
  layer3.trainable=False
  model.add(layer2)

  model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=[binary_accuracy])
  return model

#pass the mean of the encodings as inputs (size 3 input: mean_sentence_enc | mean_ontology_enc | cos_sim)
#try other activation func (tanh, leaky relu, exponential relu, softplus)
#try sgd instead of adam
#try conditional random field (crf) - instead of NN
#try 5, 10 epochs instead of 100


# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
  results = list()
  n_inputs, n_outputs = 1, 1
  # define evaluation procedure
  #cv =  ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
  cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=1)
  # enumerate folds
  for train, test in cv.split(X, y):
    # prepare data
    X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
    # define model
    model = get_model(n_inputs, n_outputs)
    # fit model
    model.fit(X_train, y_train, epochs=100, batch_size=100, verbose=0)
    # make a prediction on the test set
    yhat = model.predict(X_test)
    # round probabilities to class labels
    yhat = yhat.round()
    #yhat = (yhat > 0.5)
    # calculate accuracy
    acc = f1_score(y_test, yhat, average='micro')
    # store result
    print('>%.3f' % acc)
    results.append(acc)
    return results, model, y[train], yhat


10 epochs - F1 = 0.12
25 epochs - F1 = 0.32, AUC = 0.67
35 epochs - F1 = 0.26, AUC = 0.74
50 epochs - F1 = 0.28, AUC = 0.77
75 epochs - F1 = 0.25, AUC = 0.75

Change training metric
binary_accuracy - F1 = 0.32, AUC = 0.67
BinaryCrossEntropy: F1 = 0.3, AUC = 0.69
AUC: F1 = 0.25, AUC = 0.61

Optimizer:
adam - F1 = 0.32, AUC = 0.67
sgd - F1 = 0.1 , AUC = 0.57


Activation func:
relu  - F1 = 0.32, AUC = 0.67
tanh  - F1 = 0.24, AUC = 0.69
leakyRelu - F1 = 0.27, AUC = 0.64
ELU - F1 = 0.4, AUC = 0.66

In [ ]:
from keras.metrics import binary_accuracy
def get_model(n_inputs, n_outputs):
  model = Sequential()
  model.add(Dense(1, input_dim=n_inputs, activation='ELU'))
  model.add(Dense(n_outputs, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[binary_accuracy])
  return model

#pass the mean of the encodings as inputs (size 3 input: mean_sentence_enc | mean_ontology_enc | cos_sim)
#try other activation func (tanh, leaky relu, exponential relu, softplus)
#try sgd instead of adam
#try conditional random field (crf) - instead of NN
#try 5, 10 epochs instead of 100

# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
  results = list()
  n_inputs, n_outputs = X.shape[1], 1
  # define evaluation procedure
  #cv =  ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
  cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=1)
  # enumerate folds
  for train, test in cv.split(X, y):
    # prepare data
    X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
    # define model
    model = get_model(n_inputs, n_outputs)
    # fit model
    model.fit(X_train, y_train, epochs=25, batch_size=100, verbose=0)
    # make a prediction on the test set
    yhat = model.predict(X_test)
    # round probabilities to class labels
    yhat = yhat.round()
    #yhat = (yhat > 0.5)
    # calculate accuracy
    acc = f1_score(y_test, yhat, average='micro')
    # store result
    print('>%.3f' % acc)
    results.append(acc)
    return results, model, y[train], yhat


In [ ]:
for i, o in enumerate(ontology):
  X_cos_sim = pd.DataFrame(finalDF[finalDF['ontology']==o].iloc[:,6+i]).to_numpy()
  X_encoding = pd.DataFrame(finalDF[finalDF['ontology']==o][['encoding']]).to_numpy()
  X_ontology = np.array(finalDF[finalDF['ontology']==o].iloc[:,14+i].values.tolist()).reshape(-1,1)
  X = np.hstack((X_cos_sim,X_encoding,X_ontology))
  y = finalDF[finalDF['ontology']==o]['label'].to_numpy().reshape(-1,1)

  # evaluate model
  results_f1, model, ytrain, yhat = evaluate_model(X, y)
  # summarize performance
  print('F1: %.3f (%.3f)' % (mean(results_f1), std(results_f1)))

  model.save('/content/drive/MyDrive/DCAST/Financial/Outputs/saved_classifiers/classifier_main_model_cat' + str(i) + '.h5')

1/1 [==============================] - 0s 181ms/step
>0.364
F1: 0.364 (0.000)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2/2 [==============================] - 0s 9ms/step
>0.314
F1: 0.314 (0.000)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2/2 [==============================] - 0s 7ms/step
>0.714
F1: 0.714 (0.000)


...

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


>0.921
F1: 0.921 (0.000)
2/2 [==============================] - 0s 6ms/step
>0.477
F1: 0.477 (0.000)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3/3 [==============================] - 0s 4ms/step
>0.284
F1: 0.284 (0.000)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#model.save('/content/drive/MyDrive/DCAST/Financial/Outputs/saved_classifiers/classifier_main_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### **Encoding and Classifying test set**

In [ ]:
!pip install sentence_transformers --quiet

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util
import os

In [ ]:
test = pd.read_excel('/content/drive/MyDrive/DCAST/Financial/Inputs/8.Test_Set_Sentences_13Oct23_v1aos.xlsx')

In [ ]:
#encode test set
model_name = 'all-mpnet-base-v2-withWeights-V7-2024-01-25_00-00'
model_save_path = '/content/drive/MyDrive/DCAST/Financial/Outputs/fine_tuned_models/' + model_name


model = SentenceTransformer(model_save_path)
query_embedding = model.encode(["introduced a new generation of products",
      "extended the product range", "opened new markets",
      "entered a new technology field" ,"improved an existing product quality",
      "improved production flexibility", "reduced production cost",
      "improved yield or reduced material consumption"])

test['encoding'] = [model.encode(v) for v in test['sentence']]


In [ ]:
test = test.join(pd.DataFrame(util.cos_sim(np.array(test['encoding'].values.tolist()), query_embedding)))

In [ ]:
import pickle
import pandas as pd

In [ ]:
pickle.dump(test, open('/content/drive/MyDrive/DCAST/Financial/Outputs/encoded_test_set/' + model_name + '.pickle', 'wb'))

In [ ]:
model_name = 'all-mpnet-base-v2-withWeights-V7-2024-01-25_00-00'
test = pickle.load(open('/content/drive/MyDrive/DCAST/Financial/Outputs/encoded_test_set/' + model_name + '.pickle', 'rb'))

In [ ]:
for i in range(0, 8):
  test['ont'+str(i)] = test.apply(lambda r: mean(query_embedding[i]), axis=1)

In [ ]:
test['encoding'] = [mean(e) for e in test['encoding']]

In [ ]:
#import classifier
from keras.models import load_model

df = test

for i in range(0, 8):
  # Load model to classify category i
  model2 = load_model('/content/drive/MyDrive/DCAST/Financial/Outputs/saved_classifiers/classifier_main_model_cat' + str(i) + '.h5')
  # Define X
  X_test_cos_sim = pd.DataFrame(test.iloc[:,6+i]).to_numpy()
  X_test_encoding = pd.DataFrame(test[['encoding']]).to_numpy()
  X_test_ontology = np.array(test.iloc[:,14+i].values.tolist()).reshape(-1,1)
  X_test = np.hstack((X_test_cos_sim,X_test_encoding,X_test_ontology))
  # Predict
  yhat = model2.predict(X_test)
  #Merge with df
  df["c"+str(i+1)] = yhat


850/850 [==============================] - 1s 1ms/step


In [ ]:
#df = test.join(pd.DataFrame(yhat, columns=['c1', 'c2', 'c3', 'c4', 'c5' ,'c6' , 'c7' ,'c8']))

In [ ]:
df = df[['key', 'sentence', 'c1', 'c2', 'c3', 'c4', 'c5' ,'c6' , 'c7' ,'c8']]
df1 = pd.melt(df, id_vars = ['key', 'sentence'], var_name='category', value_name='score')
df1 = df1.iloc[:,[0,2,3]]
df1 = df1.groupby(['key', 'category'], as_index=False).max()
manual = pd.read_excel('/content/drive/MyDrive/DCAST/Financial/Inputs/9.Final_labels_agreement_firm_level_13Oct23_v1.xlsx', names=['key', 'c1', 'c2', 'c3', 'c4', 'c5' ,'c6' , 'c7' ,'c8'])
manual = pd.melt(manual, id_vars='key', var_name='category', value_name='score')
df2 = df1.merge(manual, on=['key', 'category'])
df2

,key,category,score_x,score_y
0,0000027996-17-000028,c1,0.645635,0
1,0000027996-17-000028,c2,0.545618,1
2,0000027996-17-000028,c3,0.530099,0
3,0000027996-17-000028,c4,0.482843,0
4,0000027996-17-000028,c5,0.460872,1
...,...,...,...,...
795,0001753926-20-000071,c4,0.480622,0
796,0001753926-20-000071,c5,0.461323,1
797,0001753926-20-000071,c6,0.507083,0
798,0001753926-20-000071,c7,0.622691,0


In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

overall = pd.DataFrame(columns=['Model Name', 'Weighted F-1', 'Weighted Precision', 'Weighted Recall' , 'Weighted Accuracy', 'AUC'])
f1_scoredf = pd.DataFrame(columns= ['Metric','Model','c1','c2','c3','c4','c5','c6','c7','c8'])
precision_scoredf = pd.DataFrame(columns= ['Metric','Model','c1','c2','c3','c4','c5','c6','c7','c8'])
recall_scoredf = pd.DataFrame(columns= ['Metric','Model','c1','c2','c3','c4','c5','c6','c7','c8'])
accuracy_scoredf = pd.DataFrame(columns= ['Metric','Model','c1','c2','c3','c4','c5','c6','c7','c8'])
aucdf = pd.DataFrame(columns= ['Metric','Model','c1','c2','c3','c4','c5','c6','c7','c8'])

def stat_threshold(pred, y):
  threshold = 0.5
  y_pred = [1 if x >= threshold else 0 for x in pred]
  f1 = metrics.f1_score(y, y_pred, average='binary')
  return[f1, threshold, y_pred]

#class1

y1 = df2[df2.category=='c1']['score_y']
pred1 = df2[df2.category=='c1']['score_x']
fpr1, tpr1, thresholds1 = metrics.roc_curve(y1, pred1)
f1, t1, y_pred1 = stat_threshold(pred1, y1)

#class2

y2 = df2[df2.category=='c2']['score_y']
pred2 = df2[df2.category=='c2']['score_x']
fpr2, tpr2, thresholds2 = metrics.roc_curve(y2, pred2)
f1, t2, y_pred2 = stat_threshold(pred2, y2)

#class3

y3 = df2[df2.category=='c3']['score_y']
pred3 = df2[df2.category=='c3']['score_x']
fpr3, tpr3, thresholds3 = metrics.roc_curve(y3, pred3)
f1, t3, y_pred3 = stat_threshold(pred3, y3)

#class4

y4 = df2[df2.category=='c4']['score_y']
pred4 = df2[df2.category=='c4']['score_x']
fpr4, tpr4, thresholds4 = metrics.roc_curve(y4, pred4)
f1, t4, y_pred4 = stat_threshold(pred4, y4)

#class5

y5 = df2[df2.category=='c5']['score_y']
pred5 = df2[df2.category=='c5']['score_x']
fpr5, tpr5, thresholds5 = metrics.roc_curve(y5, pred5)
f1, t5, y_pred5 = stat_threshold(pred5, y5)

#class6

y6 = df2[df2.category=='c6']['score_y']
pred6 = df2[df2.category=='c6']['score_x']
fpr6, tpr6, thresholds6 = metrics.roc_curve(y6, pred6)
f1, t6, y_pred6 = stat_threshold(pred6, y6)

#class7

y7 = df2[df2.category=='c7']['score_y']
pred7 = df2[df2.category=='c7']['score_x']
fpr7, tpr7, thresholds7 = metrics.roc_curve(y7, pred7)
f1, t7, y_pred7 = stat_threshold(pred7, y7)

#class8

y8 = df2[df2.category=='c8']['score_y']
pred8 = df2[df2.category=='c8']['score_x']
fpr8, tpr8, thresholds8 = metrics.roc_curve(y8, pred8)
f1, t8, y_pred8 = stat_threshold(pred8, y8)

#Weighted F1-score

wf1 = ((sum(y1)*metrics.f1_score(y1, y_pred1, average='binary')+ \
sum(y2)*metrics.f1_score(y2, y_pred2, average='binary')+ \
sum(y3)*metrics.f1_score(y3, y_pred3, average='binary')+ \
sum(y4)*metrics.f1_score(y4, y_pred4, average='binary')+ \
sum(y5)*metrics.f1_score(y5, y_pred5, average='binary')+ \
sum(y6)*metrics.f1_score(y6, y_pred6, average='binary')+ \
sum(y7)*metrics.f1_score(y7, y_pred7, average='binary')+ \
sum(y8)*metrics.f1_score(y8, y_pred8, average='binary'))/ \
(sum(y1) + sum(y2) + sum(y3) + sum(y4)+ \
sum(y5) + sum(y6) + sum(y7) + sum(y8)))

#Weighted Precision
wp = ((sum(y1)*metrics.precision_score(y1, y_pred1, average='binary')+
sum(y2)*metrics.precision_score(y2, y_pred2, average='binary')+ \
sum(y3)*metrics.precision_score(y3, y_pred3, average='binary')+ \
sum(y4)*metrics.precision_score(y4, y_pred4, average='binary')+ \
sum(y5)*metrics.precision_score(y5, y_pred5, average='binary')+ \
sum(y6)*metrics.precision_score(y6, y_pred6, average='binary')+ \
sum(y7)*metrics.precision_score(y7, y_pred7, average='binary')+ \
sum(y8)*metrics.precision_score(y8, y_pred8, average='binary'))/ \
(sum(y1) + sum(y2) + sum(y3) + sum(y4)+ \
sum(y5) + sum(y6) + sum(y7) + sum(y8)))

#Weighted Recall
wr = ((sum(y1)*metrics.recall_score(y1, y_pred1, average='binary')+
sum(y2)*metrics.recall_score(y2, y_pred2, average='binary')+ \
sum(y3)*metrics.recall_score(y3, y_pred3, average='binary')+ \
sum(y4)*metrics.recall_score(y4, y_pred4, average='binary')+ \
sum(y5)*metrics.recall_score(y5, y_pred5, average='binary')+ \
sum(y6)*metrics.recall_score(y6, y_pred6, average='binary')+ \
sum(y7)*metrics.recall_score(y7, y_pred7, average='binary')+ \
sum(y8)*metrics.recall_score(y8, y_pred8, average='binary'))/ \
(sum(y1) + sum(y2) + sum(y3) + sum(y4)+ \
sum(y5) + sum(y6) + sum(y7) + sum(y8)))

#Weighted accuracy
wacc = ((sum(y1)*metrics.accuracy_score(y1, y_pred1)+
sum(y2)*metrics.accuracy_score(y2, y_pred2)+ \
sum(y3)*metrics.accuracy_score(y3, y_pred3)+ \
sum(y4)*metrics.accuracy_score(y4, y_pred4)+ \
sum(y5)*metrics.accuracy_score(y5, y_pred5)+ \
sum(y6)*metrics.accuracy_score(y6, y_pred6)+ \
sum(y7)*metrics.accuracy_score(y7, y_pred7)+ \
sum(y8)*metrics.accuracy_score(y8, y_pred8))/ \
(sum(y1) + sum(y2) + sum(y3) + sum(y4)+ \
sum(y5) + sum(y6) + sum(y7) + sum(y8)))

#AUC
auc = ((sum(y1)*metrics.auc(fpr1, tpr1)+
sum(y2)*metrics.auc(fpr2, tpr2)+ \
sum(y3)*metrics.auc(fpr3, tpr3)+ \
sum(y4)*metrics.auc(fpr4, tpr4)+ \
sum(y5)*metrics.auc(fpr5, tpr5)+ \
sum(y6)*metrics.auc(fpr6, tpr6)+ \
sum(y7)*metrics.auc(fpr7, tpr7)+ \
sum(y8)*metrics.auc(fpr8, tpr8))/ \
(sum(y1) + sum(y2) + sum(y3) + sum(y4)+ \
sum(y5) + sum(y6) + sum(y7) + sum(y8)))

i=1
overall.at[i, 'Model Name'] = 'main'
overall.at[i, 'Weighted F-1'] = wf1
overall.at[i, 'Weighted Precision'] = wp
overall.at[i, 'Weighted Recall'] = wr
overall.at[i, 'Weighted Accuracy'] = wacc
overall.at[i, 'AUC'] = auc

for dataf in ['f1_score', 'precision_score', 'recall_score', 'accuracy_score', 'auc']:
  globals()[dataf + 'df'].at[i, 'Metric'] = dataf
  globals()[dataf + 'df'].at[i, 'Model'] = 'main'
  for cat in range(1,9):
    if dataf in ['f1_score', 'precision_score', 'recall_score']:
      globals()[dataf + 'df'].at[i, 'c'+ str(cat)] = globals()[dataf](globals()['y'+str(cat)], globals()['y_pred'+str(cat)], average='binary')
    elif dataf == 'accuracy':
      globals()[dataf + 'df'].at[i, 'c'+ str(cat)] = globals()[dataf](globals()['y'+str(cat)], globals()['y_pred'+str(cat)])
    else:
      globals()[dataf + 'df'].at[i, 'c'+ str(cat)] = metrics.auc(globals()['fpr'+str(cat)], globals()['tpr'+str(cat)])



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defin

In [ ]:
overall

,Model Name,Weighted F-1,Weighted Precision,Weighted Recall,Weighted Accuracy,AUC
1,main,0.37526,0.266016,0.762887,0.451753,0.522474


In [ ]:
f1_scoredf

,Metric,Model,c1,c2,c3,c4,c5,c6,c7,c8
1,f1_score,main,0.290598,0.701299,0.544118,0.0,0.0,0.188679,0.275862,0.245614


In [ ]:
####

In [ ]:
test = pickle.load(open('/content/drive/MyDrive/DCAST/Financial/Outputs/encoded_test_set/' + model_name + '.pickle', 'rb'))

In [ ]:
X_test = pd.DataFrame(test.iloc[:,4:]).to_numpy()

In [ ]:
test = test.rename(columns={0:'c1', 1:'c2', 2:'c3', 3:'c4', 4:'c5', 5:'c6', 6:'c7', 7:'c8'})

In [ ]:
df = test[['key', 'sentence', 'c1', 'c2', 'c3', 'c4', 'c5' ,'c6' , 'c7' ,'c8']]

In [ ]:
df = pd.melt(df, id_vars = ['key', 'sentence'], var_name='category', value_name='score')
df1 = df.iloc[:,[0,2,3]]
df1 = df1.groupby(['key', 'category'], as_index=False).max()

In [ ]:
manual = pd.read_excel('/content/drive/MyDrive/DCAST/Financial/Inputs/9.Final_labels_agreement_firm_level_13Oct23_v1.xlsx', names=['key', 'c1', 'c2', 'c3', 'c4', 'c5' ,'c6' , 'c7' ,'c8'])

In [ ]:
manual = pd.melt(manual, id_vars='key', var_name='category', value_name='score')
df2 = df1.merge(manual, on=['key', 'category'])

In [ ]:
df2[df2['category']=='c1']

,key,category,score_x,score_y
0,0000027996-17-000028,c1,0.799073,0
8,0000034563-17-000064,c1,0.799340,0
16,0000038725-16-000276,c1,0.753136,0
24,0000040554-14-000023,c1,0.828727,0
32,0000047307-13-000013,c1,0.860933,1
...,...,...,...,...
760,0001654954-18-002934,c1,0.693871,0
768,0001654954-19-002189,c1,0.658435,0
776,0001654954-20-002496,c1,0.499160,0
784,0001728949-18-000095,c1,0.819257,0


In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

def stat_threshold(pred, y):
  threshold = np.mean(df.score) + 3*np.std(df.score)
  y_pred = [1 if x >= threshold else 0 for x in pred]
  f1 = metrics.f1_score(y, y_pred, average='binary')
  return[f1, threshold, y_pred]

#class1

y1 = df2[df2.category=='c1']['score_y']
pred1 = df2[df2.category=='c1']['score_x']
fpr1, tpr1, thresholds1 = metrics.roc_curve(y1, pred1)
f1, t1, y_pred1 = stat_threshold(pred1, y1)

#class2

y2 = df2[df2.category=='c2']['score_y']
pred2 = df2[df2.category=='c2']['score_x']
fpr2, tpr2, thresholds2 = metrics.roc_curve(y2, pred2)
f1, t2, y_pred2 = stat_threshold(pred2, y2)

#class3

y3 = df2[df2.category=='c3']['score_y']
pred3 = df2[df2.category=='c3']['score_x']
fpr3, tpr3, thresholds3 = metrics.roc_curve(y3, pred3)
f1, t3, y_pred3 = stat_threshold(pred3, y3)

#class4

y4 = df2[df2.category=='c4']['score_y']
pred4 = df2[df2.category=='c4']['score_x']
fpr4, tpr4, thresholds4 = metrics.roc_curve(y4, pred4)
f1, t4, y_pred4 = stat_threshold(pred4, y4)

#class5

y5 = df2[df2.category=='c5']['score_y']
pred5 = df2[df2.category=='c5']['score_x']
fpr5, tpr5, thresholds5 = metrics.roc_curve(y5, pred5)
f1, t5, y_pred5 = stat_threshold(pred5, y5)

#class6

y6 = df2[df2.category=='c6']['score_y']
pred6 = df2[df2.category=='c6']['score_x']
fpr6, tpr6, thresholds6 = metrics.roc_curve(y6, pred6)
f1, t6, y_pred6 = stat_threshold(pred6, y6)

#class7

y7 = df2[df2.category=='c7']['score_y']
pred7 = df2[df2.category=='c7']['score_x']
fpr7, tpr7, thresholds7 = metrics.roc_curve(y7, pred7)
f1, t7, y_pred7 = stat_threshold(pred7, y7)

#class8

y8 = df2[df2.category=='c8']['score_y']
pred8 = df2[df2.category=='c8']['score_x']
fpr8, tpr8, thresholds8 = metrics.roc_curve(y8, pred8)
f1, t8, y_pred8 = stat_threshold(pred8, y8)

#Weighted F1-score

wf1 = ((sum(y1)*metrics.f1_score(y1, y_pred1, average='binary')+ \
sum(y2)*metrics.f1_score(y2, y_pred2, average='binary')+ \
sum(y3)*metrics.f1_score(y3, y_pred3, average='binary')+ \
sum(y4)*metrics.f1_score(y4, y_pred4, average='binary')+ \
sum(y5)*metrics.f1_score(y5, y_pred5, average='binary')+ \
sum(y6)*metrics.f1_score(y6, y_pred6, average='binary')+ \
sum(y7)*metrics.f1_score(y7, y_pred7, average='binary')+ \
sum(y8)*metrics.f1_score(y8, y_pred8, average='binary'))/ \
(sum(y1) + sum(y2) + sum(y3) + sum(y4)+ \
sum(y5) + sum(y6) + sum(y7) + sum(y8)))

#Weighted Precision
wp = ((sum(y1)*metrics.precision_score(y1, y_pred1, average='binary')+
sum(y2)*metrics.precision_score(y2, y_pred2, average='binary')+ \
sum(y3)*metrics.precision_score(y3, y_pred3, average='binary')+ \
sum(y4)*metrics.precision_score(y4, y_pred4, average='binary')+ \
sum(y5)*metrics.precision_score(y5, y_pred5, average='binary')+ \
sum(y6)*metrics.precision_score(y6, y_pred6, average='binary')+ \
sum(y7)*metrics.precision_score(y7, y_pred7, average='binary')+ \
sum(y8)*metrics.precision_score(y8, y_pred8, average='binary'))/ \
(sum(y1) + sum(y2) + sum(y3) + sum(y4)+ \
sum(y5) + sum(y6) + sum(y7) + sum(y8)))

#Weighted Recall
wr = ((sum(y1)*metrics.recall_score(y1, y_pred1, average='binary')+
sum(y2)*metrics.recall_score(y2, y_pred2, average='binary')+ \
sum(y3)*metrics.recall_score(y3, y_pred3, average='binary')+ \
sum(y4)*metrics.recall_score(y4, y_pred4, average='binary')+ \
sum(y5)*metrics.recall_score(y5, y_pred5, average='binary')+ \
sum(y6)*metrics.recall_score(y6, y_pred6, average='binary')+ \
sum(y7)*metrics.recall_score(y7, y_pred7, average='binary')+ \
sum(y8)*metrics.recall_score(y8, y_pred8, average='binary'))/ \
(sum(y1) + sum(y2) + sum(y3) + sum(y4)+ \
sum(y5) + sum(y6) + sum(y7) + sum(y8)))

#Weighted accuracy
wacc = ((sum(y1)*metrics.accuracy_score(y1, y_pred1)+
sum(y2)*metrics.accuracy_score(y2, y_pred2)+ \
sum(y3)*metrics.accuracy_score(y3, y_pred3)+ \
sum(y4)*metrics.accuracy_score(y4, y_pred4)+ \
sum(y5)*metrics.accuracy_score(y5, y_pred5)+ \
sum(y6)*metrics.accuracy_score(y6, y_pred6)+ \
sum(y7)*metrics.accuracy_score(y7, y_pred7)+ \
sum(y8)*metrics.accuracy_score(y8, y_pred8))/ \
(sum(y1) + sum(y2) + sum(y3) + sum(y4)+ \
sum(y5) + sum(y6) + sum(y7) + sum(y8)))

#AUC
auc = ((sum(y1)*metrics.auc(fpr1, tpr1)+
sum(y2)*metrics.auc(fpr2, tpr2)+ \
sum(y3)*metrics.auc(fpr3, tpr3)+ \
sum(y4)*metrics.auc(fpr4, tpr4)+ \
sum(y5)*metrics.auc(fpr5, tpr5)+ \
sum(y6)*metrics.auc(fpr6, tpr6)+ \
sum(y7)*metrics.auc(fpr7, tpr7)+ \
sum(y8)*metrics.auc(fpr8, tpr8))/ \
(sum(y1) + sum(y2) + sum(y3) + sum(y4)+ \
sum(y5) + sum(y6) + sum(y7) + sum(y8)))





TypeError: ignored

In [ ]:
overall = pd.DataFrame(columns=['Model Name', 'Weighted F-1', 'Weighted Precision', 'Weighted Recall' , 'Weighted Accuracy', 'AUC'])

i=1
overall.at[i, 'Model Name'] = 'main'
overall.at[i, 'Weighted F-1'] = wf1
overall.at[i, 'Weighted Precision'] = wp
overall.at[i, 'Weighted Recall'] = wr
overall.at[i, 'Weighted Accuracy'] = wacc
overall.at[i, 'AUC'] = auc

In [ ]:
overall

,Model Name,Weighted F-1,Weighted Precision,Weighted Recall,Weighted Accuracy,AUC
1,main,0.540103,0.542053,0.56701,0.722526,0.773473


Thresholds using the training dataset

In [ ]:
test = test[['key', 'sentence', 0,1,2,3,4,5,6,7]]
test = test.rename(columns={0:'c1', 1:'c2', 2:'c3', 3:'c4', 4:'c5', 5:'c6', 6:'c7', 7:'c8'})

In [ ]:
df = pd.melt(test, id_vars = ['key', 'sentence'], var_name='category', value_name='score')
df1 = df.iloc[:,[0,2,3]]
df1 = df1.groupby(['key', 'category'], as_index=False).max()

In [ ]:
manual = pd.read_excel('/content/drive/MyDrive/DCAST/Financial/Inputs/9.Final_labels_agreement_firm_level_13Oct23_v1.xlsx', names=['key', 'c1', 'c2', 'c3', 'c4', 'c5' ,'c6' , 'c7' ,'c8'])

In [ ]:
manual = pd.melt(manual, id_vars='key', var_name='category', value_name='score')
df2 = df1.merge(manual, on=['key', 'category'])

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

overall = pd.DataFrame(columns=['Model Name', 'Weighted F-1', 'Weighted Precision', 'Weighted Recall' , 'Weighted Accuracy', 'AUC'])
f1_scoredf = pd.DataFrame(columns= ['Metric','Model','c1','c2','c3','c4','c5','c6','c7','c8'])
precision_scoredf = pd.DataFrame(columns= ['Metric','Model','c1','c2','c3','c4','c5','c6','c7','c8'])
recall_scoredf = pd.DataFrame(columns= ['Metric','Model','c1','c2','c3','c4','c5','c6','c7','c8'])
accuracy_scoredf = pd.DataFrame(columns= ['Metric','Model','c1','c2','c3','c4','c5','c6','c7','c8'])
aucdf = pd.DataFrame(columns= ['Metric','Model','c1','c2','c3','c4','c5','c6','c7','c8'])

def stat_threshold(pred, y, key):
  threshold = thresholds_dict[key].values
  y_pred = [1 if x >= threshold else 0 for x in pred]
  f1 = metrics.f1_score(y, y_pred, average='binary')
  return[f1, threshold, y_pred]

#class1

y1 = df2[df2.category=='c1']['score_y']
pred1 = df2[df2.category=='c1']['score_x']
fpr1, tpr1, thresholds1 = metrics.roc_curve(y1, pred1)
f1, t1, y_pred1 = stat_threshold(pred1, y1, 'tm30')

#class2

y2 = df2[df2.category=='c2']['score_y']
pred2 = df2[df2.category=='c2']['score_x']
fpr2, tpr2, thresholds2 = metrics.roc_curve(y2, pred2)
f1, t2, y_pred2 = stat_threshold(pred2, y2, 'tm31')

#class3

y3 = df2[df2.category=='c3']['score_y']
pred3 = df2[df2.category=='c3']['score_x']
fpr3, tpr3, thresholds3 = metrics.roc_curve(y3, pred3)
f1, t3, y_pred3 = stat_threshold(pred3, y3, 'tm32')

#class4

y4 = df2[df2.category=='c4']['score_y']
pred4 = df2[df2.category=='c4']['score_x']
fpr4, tpr4, thresholds4 = metrics.roc_curve(y4, pred4)
f1, t4, y_pred4 = stat_threshold(pred4, y4, 'tm33')

#class5

y5 = df2[df2.category=='c5']['score_y']
pred5 = df2[df2.category=='c5']['score_x']
fpr5, tpr5, thresholds5 = metrics.roc_curve(y5, pred5)
f1, t5, y_pred5 = stat_threshold(pred5, y5, 'tm34')

#class6

y6 = df2[df2.category=='c6']['score_y']
pred6 = df2[df2.category=='c6']['score_x']
fpr6, tpr6, thresholds6 = metrics.roc_curve(y6, pred6)
f1, t6, y_pred6 = stat_threshold(pred6, y6, 'tm35')

#class7

y7 = df2[df2.category=='c7']['score_y']
pred7 = df2[df2.category=='c7']['score_x']
fpr7, tpr7, thresholds7 = metrics.roc_curve(y7, pred7)
f1, t7, y_pred7 = stat_threshold(pred7, y7, 'tm36')

#class8

y8 = df2[df2.category=='c8']['score_y']
pred8 = df2[df2.category=='c8']['score_x']
fpr8, tpr8, thresholds8 = metrics.roc_curve(y8, pred8)
f1, t8, y_pred8 = stat_threshold(pred8, y8, 'tm37')

#Weighted F1-score

wf1 = ((sum(y1)*metrics.f1_score(y1, y_pred1, average='binary')+ \
sum(y2)*metrics.f1_score(y2, y_pred2, average='binary')+ \
sum(y3)*metrics.f1_score(y3, y_pred3, average='binary')+ \
sum(y4)*metrics.f1_score(y4, y_pred4, average='binary')+ \
sum(y5)*metrics.f1_score(y5, y_pred5, average='binary')+ \
sum(y6)*metrics.f1_score(y6, y_pred6, average='binary')+ \
sum(y7)*metrics.f1_score(y7, y_pred7, average='binary')+ \
sum(y8)*metrics.f1_score(y8, y_pred8, average='binary'))/ \
(sum(y1) + sum(y2) + sum(y3) + sum(y4)+ \
sum(y5) + sum(y6) + sum(y7) + sum(y8)))

#Weighted Precision
wp = ((sum(y1)*metrics.precision_score(y1, y_pred1, average='binary')+
sum(y2)*metrics.precision_score(y2, y_pred2, average='binary')+ \
sum(y3)*metrics.precision_score(y3, y_pred3, average='binary')+ \
sum(y4)*metrics.precision_score(y4, y_pred4, average='binary')+ \
sum(y5)*metrics.precision_score(y5, y_pred5, average='binary')+ \
sum(y6)*metrics.precision_score(y6, y_pred6, average='binary')+ \
sum(y7)*metrics.precision_score(y7, y_pred7, average='binary')+ \
sum(y8)*metrics.precision_score(y8, y_pred8, average='binary'))/ \
(sum(y1) + sum(y2) + sum(y3) + sum(y4)+ \
sum(y5) + sum(y6) + sum(y7) + sum(y8)))

#Weighted Recall
wr = ((sum(y1)*metrics.recall_score(y1, y_pred1, average='binary')+
sum(y2)*metrics.recall_score(y2, y_pred2, average='binary')+ \
sum(y3)*metrics.recall_score(y3, y_pred3, average='binary')+ \
sum(y4)*metrics.recall_score(y4, y_pred4, average='binary')+ \
sum(y5)*metrics.recall_score(y5, y_pred5, average='binary')+ \
sum(y6)*metrics.recall_score(y6, y_pred6, average='binary')+ \
sum(y7)*metrics.recall_score(y7, y_pred7, average='binary')+ \
sum(y8)*metrics.recall_score(y8, y_pred8, average='binary'))/ \
(sum(y1) + sum(y2) + sum(y3) + sum(y4)+ \
sum(y5) + sum(y6) + sum(y7) + sum(y8)))

#Weighted accuracy
wacc = ((sum(y1)*metrics.accuracy_score(y1, y_pred1)+
sum(y2)*metrics.accuracy_score(y2, y_pred2)+ \
sum(y3)*metrics.accuracy_score(y3, y_pred3)+ \
sum(y4)*metrics.accuracy_score(y4, y_pred4)+ \
sum(y5)*metrics.accuracy_score(y5, y_pred5)+ \
sum(y6)*metrics.accuracy_score(y6, y_pred6)+ \
sum(y7)*metrics.accuracy_score(y7, y_pred7)+ \
sum(y8)*metrics.accuracy_score(y8, y_pred8))/ \
(sum(y1) + sum(y2) + sum(y3) + sum(y4)+ \
sum(y5) + sum(y6) + sum(y7) + sum(y8)))

#AUC
auc = ((sum(y1)*metrics.auc(fpr1, tpr1)+
sum(y2)*metrics.auc(fpr2, tpr2)+ \
sum(y3)*metrics.auc(fpr3, tpr3)+ \
sum(y4)*metrics.auc(fpr4, tpr4)+ \
sum(y5)*metrics.auc(fpr5, tpr5)+ \
sum(y6)*metrics.auc(fpr6, tpr6)+ \
sum(y7)*metrics.auc(fpr7, tpr7)+ \
sum(y8)*metrics.auc(fpr8, tpr8))/ \
(sum(y1) + sum(y2) + sum(y3) + sum(y4)+ \
sum(y5) + sum(y6) + sum(y7) + sum(y8)))

i=1
overall.at[i, 'Model Name'] = 'main'
overall.at[i, 'Weighted F-1'] = wf1
overall.at[i, 'Weighted Precision'] = wp
overall.at[i, 'Weighted Recall'] = wr
overall.at[i, 'Weighted Accuracy'] = wacc
overall.at[i, 'AUC'] = auc

for dataf in ['f1_score', 'precision_score', 'recall_score', 'accuracy_score', 'auc']:
  globals()[dataf + 'df'].at[i, 'Metric'] = dataf
  globals()[dataf + 'df'].at[i, 'Model'] = 'main'
  for cat in range(1,9):
    if dataf in ['f1_score', 'precision_score', 'recall_score']:
      globals()[dataf + 'df'].at[i, 'c'+ str(cat)] = globals()[dataf](globals()['y'+str(cat)], globals()['y_pred'+str(cat)], average='binary')
    elif dataf == 'accuracy':
      globals()[dataf + 'df'].at[i, 'c'+ str(cat)] = globals()[dataf](globals()['y'+str(cat)], globals()['y_pred'+str(cat)])
    else:
      globals()[dataf + 'df'].at[i, 'c'+ str(cat)] = metrics.auc(globals()['fpr'+str(cat)], globals()['tpr'+str(cat)])



In [ ]:
overall

,Model Name,Weighted F-1,Weighted Precision,Weighted Recall,Weighted Accuracy,AUC
1,main,0.502832,0.365632,0.943299,0.463969,0.748018


In [ ]:
f1_scoredf

,Metric,Model,c1,c2,c3,c4,c5,c6,c7,c8
1,f1_score,main,0.314815,0.729927,0.581197,0.409836,0.408163,0.21978,0.4,0.275862


In [ ]:
aucdf

,Metric,Model,c1,c2,c3,c4,c5,c6,c7,c8
1,auc,main,0.818923,0.757246,0.749893,0.677867,0.803496,0.83,0.735119,0.619601
